#1. KEY INFO

# 

# 2. BOOKING DETAILS

## 2.1. Overlapping

In [0]:
overlapping = """
WITH 
    Z AS (
        SELECT
          *,
          EXPLODE(contract_ids) contract_ids_2,
          EXPLODE(booking_ids) booking_ids_2
        FROM
          silver.revenue.occupancy
      ), 

      X AS (
        SELECT
          Z.*,
          contracts.status,
          contracts.inserted_at,
          profiles.first_name
        FROM Z
          LEFT JOIN silver.backoffice.contracts ON contracts.id = Z.contract_ids_2
          LEFT JOIN silver.backoffice.bookings ON silver.backoffice.bookings.id = z.booking_ids_2
          LEFT JOIN silver.backoffice.profiles ON silver.backoffice.bookings.profile_guest_id = backoffice.profiles.id

          WHERE true
              AND bookings.state NOT IN ('canceled', 'denied', 'retracted', 'draft')
              AND bookings.source != 'Direct Booking'
              AND bookings.apartment_id != 673 -- TEST PROPERTY
              AND (profiles.id IN (10820, 12078, 11963, 11986) 
                  OR (profiles.email != 'guests@ukio.com' OR profiles.email IS NULL))
      ),

      S as (
        SELECT
          COUNT(day) c,
          booking_ids_2,
          day
        FROM X
        WHERE first_name NOT ILIKE '%test%' AND contract_ids_2 NOT IN (75120) OR contract_ids_2 IS NULL
        GROUP BY
          booking_ids_2,
          day
        ORDER BY
          COUNT(day) DESC
      ),

      Y AS (
        SELECT
          S.*,
          x.contract_ids_2,
          x.inserted_at
        FROM
          S
          LEFT JOIN x ON s.day = x.day
          AND s.booking_ids_2 = x.booking_ids_2
        WHERE
          c = 2
        ORDER BY
          x.booking_ids_2 DESC,
          x.day DESC
      ),
      A AS (
        SELECT
          *,
          ROW_NUMBER() OVER (
            PARTITION BY booking_ids_2,
            day
            ORDER BY
              inserted_at ASC
          ) AS RN
        FROM
          Y
        ORDER BY
          booking_ids_2 DESC,
          day DESC
      ),

      B AS (
        SELECT
          DISTINCT x.*,
          coalesce(a.rn, 2) rn
        FROM
          X
          LEFT JOIN A ON X.day = a.day
          AND x.contract_ids_2 = a.contract_ids_2
          AND x.inserted_at = a.inserted_at --WHERE rn = 2
      ),

      C AS (
        SELECT
          contract_ids_2,
          booking_ids_2,
          day,
          APARTMENT_ID,
          IS_BLOCKED,
          block_reason,
          IS_BOOKED
        FROM
          B
        WHERE
          rn = 2
        ORDER BY
          day DESC
      ),

      D AS (
        SELECT
          *,
          EXPLODE(booking_ids) booking_ids_2
        FROM silver.revenue.occupancy

        ORDER BY
          day DESC
      ),

      E_2 AS (
        SELECT
          COALESCE(C.contract_ids_2, NULL) CONTRACT_ID,
          D.booking_ids_2,
          D.APARTMENT_ID,
          D.DAY,
          D.IS_BLOCKED,
          D.block_reason,
          D.IS_BOOKED, first_name
        FROM D
          LEFT JOIN C ON D.booking_ids_2 = C.booking_ids_2 AND D.DAY = C.DAY AND D.APARTMENT_ID = C.APARTMENT_ID
          LEFT JOIN silver.backoffice.bookings ON silver.backoffice.bookings.id = d.booking_ids_2
          LEFT JOIN silver.backoffice.profiles ON silver.backoffice.bookings.profile_guest_id = backoffice.profiles.id
        WHERE true
          AND contract_ids_2 NOT IN (75120) OR contract_ids_2 IS NULL
          AND bookings.state NOT IN ('canceled', 'denied', 'retracted', 'draft')
          AND bookings.source != 'Direct Booking'
          AND bookings.apartment_id != 673 -- TEST PROPERTY
          AND (profiles.id IN (10820, 12078, 11963, 11986) 
              OR (profiles.email != 'guests@ukio.com' OR profiles.email IS NULL))
      ),

      E AS (
        SELECT
          *,
          CASE
            WHEN day <= MIN(day) OVER (PARTITION BY booking_ids_2, APARTMENT_ID) THEN MIN(CONTRACT_ID) OVER (PARTITION BY booking_ids_2, APARTMENT_ID)
            ELSE MAX(CONTRACT_ID) OVER (PARTITION BY booking_ids_2, APARTMENT_ID)
          END AS Z
        FROM E_2
        WHERE first_name NOT ILIKE '%test%' 
        ORDER BY
          DAY DESC
      ),

      F AS (
        SELECT
          booking_ids_2,
          APARTMENT_ID,
          DAY,
          IS_BLOCKED,
          block_reason,
          IS_BOOKED,
          COALESCE(CONTRACT_ID, Z) AS CONTRACT_ID_FILLED
        FROM
          E
      ),

      G AS (
        SELECT
          F.*,
          CASE
            WHEN contracts.inserted_at > bookings.check_in THEN bookings.check_in
            ELSE contracts.inserted_at
          END AS inserted_at,
          apartments.city
        FROM
          F
          LEFT JOIN silver.backoffice.contracts ON contracts.id = F.CONTRACT_ID_FILLED
          LEFT JOIN silver.backoffice.apartments ON apartments.id = F.APARTMENT_ID
          LEFT JOIN silver.backoffice.bookings ON bookings.id = contracts.booking_id
      )

  --SELECT * FROM E_2 WHERE booking_ids_2 = 12049

    SELECT
      g.CONTRACT_ID_FILLED,
      g.inserted_at,
      MAX(g.day) + 1 co,
      MIN(g.day) ci,
      g.booking_ids_2,
      g.city,
      contracts.start_date,
      contracts.end_date
    FROM
      G
      LEFT JOIN silver.backoffice.contracts ON contracts.id = g.CONTRACT_ID_FILLED  
    group by
      g.CONTRACT_ID_FILLED,
      g.inserted_at,
      g.booking_ids_2,
      g.city,
      contracts.start_date,
      contracts.end_date
"""

spark.sql(overlapping).write.mode("overwrite").saveAsTable("silver.finance.overlapping")


## 2.2.Bookings

### 2.2.1. Margins per booking

#### 2.2.1.1. Actuals 
###### (Margins per booking) 

In [0]:
margins_per_booking = """
with 
  payments AS (
      SELECT
        ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
        b.id AS booking_id,
        contracts.inserted_at :: DATE AS sell_date,
        -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        contracts.id as contract_id,
        CASE
          WHEN contracts.end_date > b.check_out THEN b.check_out --handle early departure check-out dates
          WHEN contracts.source = 'Hostify' THEN b.check_out --handle Hostify extensions check-out dates
          ELSE contracts.end_date
        END AS check_out,
        transactions.id as transaction_id,
        transaction_items.description,
        case
          when contract_lines.code = 'service_charge'
          and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price :: decimal / 100 :: decimal)
          else (transaction_items.price :: decimal / 100 :: decimal)
        end as payment
      from
        silver.backoffice.bookings b
        left join silver.backoffice.contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft')
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND transactions.status not in ('canceled', 'expired', 'discarded')
        AND contracts.source NOT ILIKE '%extension%' --and transaction_items.price > 0
        and (transaction_items.description ILIKE '%rent%')
    ),
    ttl_rent AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        sell_date,
        contract_id,
        SUM(payment) AS total_rent
      FROM
        payments
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        contract_id
    ),
    comm AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN total_rent / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN total_rent / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN total_rent / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN total_rent / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN total_rent / 1.15 --otas
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN total_rent / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN total_rent / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN total_rent / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_rent / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Berlin', 'Paris') THEN total_rent / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN total_rent / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN total_rent / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN total_rent / 1.15
          ELSE total_rent
        END AS total_rent,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        sell_date,
        contract_id
      FROM
        ttl_rent
    ),
    remove_vat AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        --to avoid divisions by zero I'm adding a case when in the column of nights
        case
          when nights :: integer = 0 then 1
          else nights :: integer
        end as nights,
        source,
        sell_date,
        contract_id,
        case
          when city = 'Madrid'
          or city = 'Barcelona' then total_rent
          when city = 'Berlin' then total_rent
          else total_rent
        end as total_rent
      FROM
        comm
      WHERE
        true --AND check_out - check_in >= 365
        --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
      ORDER BY
        booking_id DESC
    ) ---------------------------------------------UNTIL HERE RENT----------------------------------------------------
  ,
    payments_sc AS (
      SELECT
        ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
        b.id AS booking_id,
        contracts.inserted_at :: DATE AS sell_date,
        -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        contracts.end_date as check_out,
        transactions.id as transaction_id,
        transaction_items.description,
        case
          when contract_lines.code = 'service_charge'
          and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price :: decimal / 100 :: decimal)
          else (transaction_items.price :: decimal / 100 :: decimal)
        end as payment
      from
        silver.backoffice.bookings b
        left join silver.backoffice.contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft') -------------not including canceled, since no margin gets generated--------
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND transactions.status not in ('canceled', 'expired', 'discarded')
        AND contracts.source NOT ILIKE '%extension%' --and transaction_items.price > 0
        and (
          contract_lines.code = 'service_charge'
          or (
            transaction_items.description ILIKE '%service charge%'
          )
          or (
            transaction_items.description ILIKE '%service charge%'
          ) -----------------check this------------------ops discounts, refunds
          or (
            transaction_items.description ILIKE '%service_charge%'
          )
          or (
            transaction_items.description ILIKE '%administration fee%'
          ) ----------------check---------------including refunds, penalties
          or (
            transaction_items.description ILIKE '%admini fee%'
          )
        )
    ),
    ttl_rent_sc AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        SUM(payment) AS total_sc
      FROM
        payments_sc
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        state
    ),
    comm_sc AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN total_sc / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN total_sc / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN total_sc / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN total_sc / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN total_sc / 1.15 --otas
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN total_sc / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN total_sc / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN total_sc / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_sc / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Berlin', 'Paris') THEN total_sc / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN total_sc / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN total_sc / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN total_sc / 1.15
          ELSE total_sc
        END AS total_sc,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date
      FROM
        ttl_rent_sc
    ),
    remove_vat_sc AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        case
          when city = 'Madrid'
          or city = 'Barcelona' then total_sc
          when city = 'Berlin' then total_sc
          else total_sc
        end as total_sc
      FROM
        comm_sc
      WHERE
        true --AND check_out - check_in >= 365
        --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
      ORDER BY
        booking_id DESC
    ) ---------------------------------------------UNTIL HERE SC----------------------------------------------------
  ,
    full_revenue AS (
      SELECT
        b.id AS booking_id,
        b.inserted_at :: DATE AS sell_date,
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        CASE
          WHEN contracts.end_date > b.check_out THEN b.check_out --handle early departure check-out dates
          WHEN contracts.source = 'Hostify' THEN b.check_out --handle Hostify extensions check-out dates
          ELSE contracts.end_date
        END AS check_out,
        CASE
          WHEN contract_lines.code = 'utilities'
          OR contract_lines.code = 'building_utilities' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS utilities,
        CASE
          WHEN contract_lines.code = 'final_cleaning' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS final_cleaning,
        CASE
          WHEN contract_lines.code = 'pets' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS pets,
        CASE
          WHEN contract_lines.code ILIKE '%extra_cleaning%' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS extra_cleaning,
        CASE
          WHEN contract_lines.code = 'airport_transport' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS airport_transport,
        CASE
          WHEN contract_lines.code = 'in_person_check_in' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS in_person_check_in,
        CASE
          WHEN contract_lines.code = 'parking' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS parking,
        CASE
          WHEN contract_lines.code = 'deposit' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS deposit,
        CASE
          WHEN contract_lines.code NOT IN (
            'rent',
            'service_charge',
            'utilities',
            'building_utilities',
            'final_cleaning',
            'pets',
            'airport_transport',
            'in_person_check_in',
            'parking',
            'deposit'
          )
          AND contract_lines.code NOT ILIKE '%extra_cleaning%' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS other_revenue
      from
        silver.backoffice.bookings b
        left join silver.backoffice.contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft')
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND contracts.source NOT ILIKE '%extension%'
        AND transactions.status not in ('canceled', 'expired', 'discarded')
    ),
    ttl_revenue AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        SUM(utilities) AS utilities,
        SUM(final_cleaning) AS final_cleaning,
        SUM(pets) AS pets,
        SUM(extra_cleaning) AS extra_cleaning,
        SUM(in_person_check_in) AS in_person_check_in,
        SUM(parking) AS parking,
        SUM(deposit) AS deposit,
        SUM(other_revenue) AS other_revenue
      FROM
        full_revenue
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        state
    ),
    comm_rev AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN utilities / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN utilities / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN utilities / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN utilities / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN utilities / 1.15 --otas (all except HA)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN utilities / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN utilities / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN utilities / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN utilities / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN utilities / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN utilities / 1.15
          ELSE utilities
        END AS utilities,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN final_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN final_cleaning / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN final_cleaning / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN final_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN final_cleaning / 1.15 --otas (only Airbnb)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN final_cleaning / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN final_cleaning / 1.067
          ELSE final_cleaning
        END AS final_cleaning,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN pets / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN pets / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN pets / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN pets / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN pets / 1.15 --otas (only Airbnb)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN pets / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN pets / 1.067
          ELSE pets
        END AS pets,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN extra_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN extra_cleaning / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN extra_cleaning / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN extra_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN extra_cleaning / 1.15
          ELSE extra_cleaning
        END AS extra_cleaning,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN in_person_check_in / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN in_person_check_in / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN in_person_check_in / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN in_person_check_in / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN in_person_check_in / 1.15
          ELSE in_person_check_in
        END AS in_person_check_in,
        other_revenue,
        parking,
        --not commissionable
        deposit,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date
      FROM
        ttl_revenue
    ),
    rev_wo_vat AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        utilities,
        final_cleaning,
        pets,
        extra_cleaning,
        in_person_check_in,
        parking,
        deposit,
        other_revenue
      FROM
        comm_rev
      WHERE
        true
      ORDER BY
        booking_id DESC
    ) ------------------------------------UNTIL HERE FULL REVENUE-----------------------------------------------------------
  ,
    landlord_cogs_2 AS (
      SELECT
        SUM(landlord_daily_rent) :: decimal as landlord,
        trunc(day, 'month') as month,
        end_of_grace_period :: DATE + 1 as first_landlord_rent_payment_date,
        codename,
        city
      FROM
        silver.revenue.landlord
        LEFT JOIN silver.Backoffice.apartments ON silver.Backoffice.apartments.id = silver.revenue.landlord.apartment_id
      GROUP BY
        month,
        codename,
        end_of_grace_period,
        city
    ),
    landlord_cogs AS (
      SELECT
        codename,
        city,
        month,
        landlord,
        CASE
          WHEN trunc(first_landlord_rent_payment_date, 'MM') = trunc(month, 'MM') THEN (
            trunc(month, 'MM') + INTERVAL '1' MONTH - INTERVAL '1' DAY - first_landlord_rent_payment_date :: DATE
          ) :: INT
          ELSE EXTRACT(
            DAY
            FROM
              trunc(month, 'mm') + INTERVAL '1' MONTH - INTERVAL '1' DAY
          ) :: INT
        END AS x,
        first_landlord_rent_payment_date
      FROM
        landlord_cogs_2
    ),
    split_month AS (
      SELECT
        codename,
        booking_id,
        city,
        check_in,
        check_out,
        explode(
          sequence(
            trunc(check_in :: date, 'month'),
            trunc(check_out :: date, 'month'),
            INTERVAL 1 month
          )
        ) AS month
      FROM
        rev_wo_vat
    ),
    ll_per_booking_2 AS (
      SELECT
        s.codename,
        s.city,
        s.booking_id,
        S.MONTH,
        CASE
          WHEN s.MONTH = trunc(check_in, 'month')
          AND s.MONTH = date_trunc('month', check_out) THEN (check_out :: DATE - check_in :: DATE) :: INT
          WHEN s.MONTH = trunc(check_in, 'month') THEN (
            trunc(check_in, 'month') + interval '1 month' - check_in :: DATE
          ) :: INT
          WHEN s.MONTH = date_trunc('month', check_out) THEN (
            check_out :: DATE - date_trunc('month', check_out) :: DATE
          ) :: INT
          ELSE (
            (trunc(s.MONTH, 'month') + interval '1 month') :: DATE - s.MONTH :: DATE
          ) :: INT
        END AS days
      FROM
        split_month s
    ),
    ll_per_booking AS (
      SELECT
        s.codename,
        s.city,
        s.booking_id,
        SUM(
          CASE
            WHEN trunc(first_landlord_rent_payment_date, 'month') = trunc(s.month, 'month')
            AND X <= days THEN X
            ELSE days
          END * (
            landlord :: DECIMAL / CASE
              WHEN X = 0 THEN 1
              ELSE X
            END
          )
        ) AS Landlord
      FROM
        ll_per_booking_2 s
        LEFT JOIN landlord_cogs l ON s.MONTH = l.MONTH
        AND s.codename = l.codename
      GROUP BY
        s.codename,
        s.city,
        s.booking_id
      ORDER BY
        s.booking_id DESC
    ) --------------------------------------UNTIL HERE LANDLORD, CONSIDERING GRD AND CARENCIA-----------------
  ,
    final_2 AS (
      SELECT
        remove_vat.*,
        rev_wo_vat.utilities,
        rev_wo_vat.final_cleaning,
        rev_wo_vat.pets,
        rev_wo_vat.extra_cleaning,
        rev_wo_vat.in_person_check_in,
        rev_wo_vat.parking,
        rev_wo_vat.deposit,
        rev_wo_vat.other_revenue,
        remove_vat_sc.total_sc,
        ll_per_booking.Landlord AS total_landlord
      FROM
        remove_vat
        LEFT JOIN rev_wo_vat ON rev_wo_vat.booking_id = remove_vat.booking_id
        LEFT JOIN remove_vat_sc ON rev_wo_vat.booking_id = remove_vat_sc.booking_id
        LEFT JOIN ll_per_booking ON rev_wo_vat.booking_id = ll_per_booking.booking_id
    ),
    final AS (
      SELECT
        *,
        COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0) AS total_rev,
        total_landlord / nights *(365 / 12) AS avg_monthly_landlord,
        case
          ---VAT of total_revenue
          when city = 'Madrid'
          or city = 'Barcelona' then (
            COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.1 * 0.1
          when city = 'Berlin' then (
            COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.07 * 0.07
          else 0
        end as vat_revenue,
        case
          ---VAT of total_rent
          when city = 'Madrid'
          or city = 'Barcelona' then total_rent / 1.1 * 0.1
          when city = 'Berlin' then total_rent / 1.07 * 0.07
          else 0
        end as vat_rent,
        case
          ---VAT of total_sc
          when city = 'Madrid'
          or city = 'Barcelona' then total_sc / 1.1 * 0.1
          when city = 'Berlin' then total_sc / 1.07 * 0.07
          else 0
        end as vat_sc,
        case
          ---VAT of other revenue concepts
          when city = 'Madrid'
          or city = 'Barcelona' then (
            COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.1 * 0.1
          when city = 'Berlin' then (
            COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.07 * 0.07
          else 0
        end as vat_other_rev
      FROM
        final_2
    ),
    Est AS (
      SELECT
        k.contract_id,
        cl.code,
        (cl.price :: DECIMAL / 100) AS SC_Price
      FROM
        final k
        LEFT JOIN silver.backoffice.contract_lines cl ON k.contract_id = cl.contract_id
      WHERE
        code = 'service_charge'
        AND k.city = 'Paris'
        AND k.sell_date >= DATE('2023-12-04')
    ),
    X AS (
      SELECT
        k.codename,
        k.booking_id,
        k.city,
        k.check_in,
        k.check_out,
        k.source,
        k.sell_date,
        k.contract_id,
        k.nights,
        k.total_rent,
        CASE
          WHEN (
            city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
            AND k.total_sc IS NULL
          ) THEN utilities - (
            CASE
              WHEN SC_Price IS NULL THEN 0
              ELSE SC_Price
            END
          )
          ELSE k.utilities
        END AS utilities,
        CASE
          WHEN (
            city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
            AND k.total_sc IS NULL
          ) THEN (
            CASE
              WHEN SC_Price IS NULL THEN 0
              ELSE SC_Price
            END
          )
          ELSE k.total_sc
        END AS total_sc,
        k.final_cleaning,
        k.pets,
        k.extra_cleaning,
        k.in_person_check_in,
        k.parking,
        k.deposit,
        k.other_revenue,
        k.total_landlord,
        k.total_rev,
        k.avg_monthly_landlord,
        k.vat_revenue,
        k.vat_rent,
        k.vat_sc,
        k.vat_other_rev
      FROM
        final k
        LEFT JOIN Est ON k.contract_id = Est.contract_id
    ),
    k_i AS (
      SELECT
        *,
        COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent - total_landlord AS total_margin,
        (
          (COALESCE(total_rent, 0) + COALESCE(total_sc, 0)) - vat_sc - vat_rent - total_landlord
        ) /(
          CASE
            WHEN nights = 0 THEN 1
            ELSE nights
          END
        ) * 365 / 12 AS mg_booked,
        (
          (COALESCE(total_rent, 0) + COALESCE(total_sc, 0)) - vat_sc - vat_rent - total_landlord
        ) / (
          CASE
            WHEN (
              COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent
            ) = 0 THEN 1
            ELSE (
              COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent
            )
          END
        ) AS pct_mg
      FROM
        X
      WHERE
        true
      ORDER BY
        booking_id DESC
    )
  SELECT
    *
  FROM
    k_i
"""

spark.sql(margins_per_booking).write.mode("overwrite").saveAsTable("silver.finance.margins_per_booking")

#### 2.2.1.2. Deltas 7 Days Ago
###### (Margins per booking) 

In [0]:
margins_per_booking_7_days_deltas = """
with 
  payments AS (
      SELECT
        ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
        b.id AS booking_id,
        contracts.inserted_at :: DATE AS sell_date,
        -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        contracts.id as contract_id,
        CASE
          WHEN contracts.end_date > b.check_out THEN b.check_out --handle early departure check-out dates
          WHEN contracts.source = 'Hostify' THEN b.check_out --handle Hostify extensions check-out dates
          ELSE contracts.end_date
        END AS check_out,
        transactions.id as transaction_id,
        transaction_items.description,
        case
          when contract_lines.code = 'service_charge'
          and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price :: decimal / 100 :: decimal)
          else (transaction_items.price :: decimal / 100 :: decimal)
        end as payment
      from
        silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b
        left join silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft')
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND transactions.status not in ('canceled', 'expired', 'discarded')
        AND contracts.source NOT ILIKE '%extension%' --and transaction_items.price > 0
        and (transaction_items.description ILIKE '%rent%')
    ),
    ttl_rent AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        sell_date,
        contract_id,
        SUM(payment) AS total_rent
      FROM
        payments
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        contract_id
    ),
    comm AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN total_rent / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN total_rent / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN total_rent / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN total_rent / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN total_rent / 1.15 --otas
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN total_rent / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN total_rent / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN total_rent / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_rent / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Berlin', 'Paris') THEN total_rent / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN total_rent / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN total_rent / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN total_rent / 1.15
          ELSE total_rent
        END AS total_rent,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        sell_date,
        contract_id
      FROM
        ttl_rent
    ),
    remove_vat AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        --to avoid divisions by zero I'm adding a case when in the column of nights
        case
          when nights :: integer = 0 then 1
          else nights :: integer
        end as nights,
        source,
        sell_date,
        contract_id,
        case
          when city = 'Madrid'
          or city = 'Barcelona' then total_rent
          when city = 'Berlin' then total_rent
          else total_rent
        end as total_rent
      FROM
        comm
      WHERE
        true --AND check_out - check_in >= 365
        --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
      ORDER BY
        booking_id DESC
    ) ---------------------------------------------UNTIL HERE RENT----------------------------------------------------
  ,
    payments_sc AS (
      SELECT
        ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
        b.id AS booking_id,
        contracts.inserted_at :: DATE AS sell_date,
        -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        contracts.end_date as check_out,
        transactions.id as transaction_id,
        transaction_items.description,
        case
          when contract_lines.code = 'service_charge'
          and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price :: decimal / 100 :: decimal)
          else (transaction_items.price :: decimal / 100 :: decimal)
        end as payment
      from
        silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b
        left join silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft') -------------not including canceled, since no margin gets generated--------
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND transactions.status not in ('canceled', 'expired', 'discarded')
        AND contracts.source NOT ILIKE '%extension%' --and transaction_items.price > 0
        and (
          contract_lines.code = 'service_charge'
          or (
            transaction_items.description ILIKE '%service charge%'
          )
          or (
            transaction_items.description ILIKE '%service charge%'
          ) -----------------check this------------------ops discounts, refunds
          or (
            transaction_items.description ILIKE '%service_charge%'
          )
          or (
            transaction_items.description ILIKE '%administration fee%'
          ) ----------------check---------------including refunds, penalties
          or (
            transaction_items.description ILIKE '%admini fee%'
          )
        )
    ),
    ttl_rent_sc AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        SUM(payment) AS total_sc
      FROM
        payments_sc
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        state
    ),
    comm_sc AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN total_sc / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN total_sc / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN total_sc / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN total_sc / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN total_sc / 1.15 --otas
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN total_sc / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN total_sc / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN total_sc / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_sc / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Berlin', 'Paris') THEN total_sc / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN total_sc / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN total_sc / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN total_sc / 1.15
          ELSE total_sc
        END AS total_sc,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date
      FROM
        ttl_rent_sc
    ),
    remove_vat_sc AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        case
          when city = 'Madrid'
          or city = 'Barcelona' then total_sc
          when city = 'Berlin' then total_sc
          else total_sc
        end as total_sc
      FROM
        comm_sc
      WHERE
        true --AND check_out - check_in >= 365
        --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
      ORDER BY
        booking_id DESC
    ) ---------------------------------------------UNTIL HERE SC----------------------------------------------------
  ,
    full_revenue AS (
      SELECT
        b.id AS booking_id,
        b.inserted_at :: DATE AS sell_date,
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        CASE
          WHEN contracts.end_date > b.check_out THEN b.check_out --handle early departure check-out dates
          WHEN contracts.source = 'Hostify' THEN b.check_out --handle Hostify extensions check-out dates
          ELSE contracts.end_date
        END AS check_out,
        CASE
          WHEN contract_lines.code = 'utilities'
          OR contract_lines.code = 'building_utilities' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS utilities,
        CASE
          WHEN contract_lines.code = 'final_cleaning' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS final_cleaning,
        CASE
          WHEN contract_lines.code = 'pets' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS pets,
        CASE
          WHEN contract_lines.code ILIKE '%extra_cleaning%' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS extra_cleaning,
        CASE
          WHEN contract_lines.code = 'airport_transport' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS airport_transport,
        CASE
          WHEN contract_lines.code = 'in_person_check_in' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS in_person_check_in,
        CASE
          WHEN contract_lines.code = 'parking' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS parking,
        CASE
          WHEN contract_lines.code = 'deposit' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS deposit,
        CASE
          WHEN contract_lines.code NOT IN (
            'rent',
            'service_charge',
            'utilities',
            'building_utilities',
            'final_cleaning',
            'pets',
            'airport_transport',
            'in_person_check_in',
            'parking',
            'deposit'
          )
          AND contract_lines.code NOT ILIKE '%extra_cleaning%' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS other_revenue
      from
        silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b
        left join silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft')
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND contracts.source NOT ILIKE '%extension%'
        AND transactions.status not in ('canceled', 'expired', 'discarded')
    ),
    ttl_revenue AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        SUM(utilities) AS utilities,
        SUM(final_cleaning) AS final_cleaning,
        SUM(pets) AS pets,
        SUM(extra_cleaning) AS extra_cleaning,
        SUM(in_person_check_in) AS in_person_check_in,
        SUM(parking) AS parking,
        SUM(deposit) AS deposit,
        SUM(other_revenue) AS other_revenue
      FROM
        full_revenue
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        state
    ),
    comm_rev AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN utilities / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN utilities / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN utilities / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN utilities / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN utilities / 1.15 --otas (all except HA)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN utilities / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN utilities / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN utilities / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN utilities / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN utilities / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN utilities / 1.15
          ELSE utilities
        END AS utilities,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN final_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN final_cleaning / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN final_cleaning / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN final_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN final_cleaning / 1.15 --otas (only Airbnb)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN final_cleaning / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN final_cleaning / 1.067
          ELSE final_cleaning
        END AS final_cleaning,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN pets / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN pets / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN pets / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN pets / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN pets / 1.15 --otas (only Airbnb)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN pets / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN pets / 1.067
          ELSE pets
        END AS pets,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN extra_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN extra_cleaning / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN extra_cleaning / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN extra_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN extra_cleaning / 1.15
          ELSE extra_cleaning
        END AS extra_cleaning,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN in_person_check_in / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN in_person_check_in / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN in_person_check_in / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN in_person_check_in / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN in_person_check_in / 1.15
          ELSE in_person_check_in
        END AS in_person_check_in,
        other_revenue,
        parking,
        --not commissionable
        deposit,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date
      FROM
        ttl_revenue
    ),
    rev_wo_vat AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        utilities,
        final_cleaning,
        pets,
        extra_cleaning,
        in_person_check_in,
        parking,
        deposit,
        other_revenue
      FROM
        comm_rev
      WHERE
        true
      ORDER BY
        booking_id DESC
    ) ------------------------------------UNTIL HERE FULL REVENUE-----------------------------------------------------------
  ,
    landlord_cogs_2 AS (
      SELECT
        SUM(landlord_daily_rent) :: decimal as landlord,
        trunc(day, 'month') as month,
        end_of_grace_period :: DATE + 1 as first_landlord_rent_payment_date,
        codename,
        city
      FROM
        silver.revenue.landlord TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) landlord
        LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON apartments.id = landlord.apartment_id
      GROUP BY
        month,
        codename,
        end_of_grace_period,
        city
    ),
    landlord_cogs AS (
      SELECT
        codename,
        city,
        month,
        landlord,
        CASE
          WHEN trunc(first_landlord_rent_payment_date, 'MM') = trunc(month, 'MM') THEN (
            trunc(month, 'MM') + INTERVAL '1' MONTH - INTERVAL '1' DAY - first_landlord_rent_payment_date :: DATE
          ) :: INT
          ELSE EXTRACT(
            DAY
            FROM
              trunc(month, 'mm') + INTERVAL '1' MONTH - INTERVAL '1' DAY
          ) :: INT
        END AS x,
        first_landlord_rent_payment_date
      FROM
        landlord_cogs_2
    ),
    split_month AS (
      SELECT
        codename,
        booking_id,
        city,
        check_in,
        check_out,
        explode(
          sequence(
            trunc(check_in :: date, 'month'),
            trunc(check_out :: date, 'month'),
            INTERVAL 1 month
          )
        ) AS month
      FROM
        rev_wo_vat
    ),
    ll_per_booking_2 AS (
      SELECT
        s.codename,
        s.city,
        s.booking_id,
        S.MONTH,
        CASE
          WHEN s.MONTH = trunc(check_in, 'month')
          AND s.MONTH = date_trunc('month', check_out) THEN (check_out :: DATE - check_in :: DATE) :: INT
          WHEN s.MONTH = trunc(check_in, 'month') THEN (
            trunc(check_in, 'month') + interval '1 month' - check_in :: DATE
          ) :: INT
          WHEN s.MONTH = date_trunc('month', check_out) THEN (
            check_out :: DATE - date_trunc('month', check_out) :: DATE
          ) :: INT
          ELSE (
            (trunc(s.MONTH, 'month') + interval '1 month') :: DATE - s.MONTH :: DATE
          ) :: INT
        END AS days
      FROM
        split_month s
    ),
    ll_per_booking AS (
      SELECT
        s.codename,
        s.city,
        s.booking_id,
        SUM(
          CASE
            WHEN trunc(first_landlord_rent_payment_date, 'month') = trunc(s.month, 'month')
            AND X <= days THEN X
            ELSE days
          END * (
            landlord :: DECIMAL / CASE
              WHEN X = 0 THEN 1
              ELSE X
            END
          )
        ) AS Landlord
      FROM
        ll_per_booking_2 s
        LEFT JOIN landlord_cogs l ON s.MONTH = l.MONTH
        AND s.codename = l.codename
      GROUP BY
        s.codename,
        s.city,
        s.booking_id
      ORDER BY
        s.booking_id DESC
    ) --------------------------------------UNTIL HERE LANDLORD, CONSIDERING GRD AND CARENCIA-----------------
  ,
    final_2 AS (
      SELECT
        remove_vat.*,
        rev_wo_vat.utilities,
        rev_wo_vat.final_cleaning,
        rev_wo_vat.pets,
        rev_wo_vat.extra_cleaning,
        rev_wo_vat.in_person_check_in,
        rev_wo_vat.parking,
        rev_wo_vat.deposit,
        rev_wo_vat.other_revenue,
        remove_vat_sc.total_sc,
        ll_per_booking.Landlord AS total_landlord
      FROM
        remove_vat
        LEFT JOIN rev_wo_vat ON rev_wo_vat.booking_id = remove_vat.booking_id
        LEFT JOIN remove_vat_sc ON rev_wo_vat.booking_id = remove_vat_sc.booking_id
        LEFT JOIN ll_per_booking ON rev_wo_vat.booking_id = ll_per_booking.booking_id
    ),
    final AS (
      SELECT
        *,
        COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0) AS total_rev,
        total_landlord / nights *(365 / 12) AS avg_monthly_landlord,
        case
          ---VAT of total_revenue
          when city = 'Madrid'
          or city = 'Barcelona' then (
            COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.1 * 0.1
          when city = 'Berlin' then (
            COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.07 * 0.07
          else 0
        end as vat_revenue,
        case
          ---VAT of total_rent
          when city = 'Madrid'
          or city = 'Barcelona' then total_rent / 1.1 * 0.1
          when city = 'Berlin' then total_rent / 1.07 * 0.07
          else 0
        end as vat_rent,
        case
          ---VAT of total_sc
          when city = 'Madrid'
          or city = 'Barcelona' then total_sc / 1.1 * 0.1
          when city = 'Berlin' then total_sc / 1.07 * 0.07
          else 0
        end as vat_sc,
        case
          ---VAT of other revenue concepts
          when city = 'Madrid'
          or city = 'Barcelona' then (
            COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.1 * 0.1
          when city = 'Berlin' then (
            COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.07 * 0.07
          else 0
        end as vat_other_rev
      FROM
        final_2
    ),
    Est AS (
      SELECT
        k.contract_id,
        cl.code,
        (cl.price :: DECIMAL / 100) AS SC_Price
      FROM
        final k
        LEFT JOIN silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) cl ON k.contract_id = cl.contract_id
      WHERE
        code = 'service_charge'
        AND k.city = 'Paris'
        AND k.sell_date >= DATE('2023-12-04')
    ),
    X AS (
      SELECT
        k.codename,
        k.booking_id,
        k.city,
        k.check_in,
        k.check_out,
        k.source,
        k.sell_date,
        k.contract_id,
        k.nights,
        k.total_rent,
        CASE
          WHEN (
            city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
            AND k.total_sc IS NULL
          ) THEN utilities - (
            CASE
              WHEN SC_Price IS NULL THEN 0
              ELSE SC_Price
            END
          )
          ELSE k.utilities
        END AS utilities,
        CASE
          WHEN (
            city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
            AND k.total_sc IS NULL
          ) THEN (
            CASE
              WHEN SC_Price IS NULL THEN 0
              ELSE SC_Price
            END
          )
          ELSE k.total_sc
        END AS total_sc,
        k.final_cleaning,
        k.pets,
        k.extra_cleaning,
        k.in_person_check_in,
        k.parking,
        k.deposit,
        k.other_revenue,
        k.total_landlord,
        k.total_rev,
        k.avg_monthly_landlord,
        k.vat_revenue,
        k.vat_rent,
        k.vat_sc,
        k.vat_other_rev
      FROM
        final k
        LEFT JOIN Est ON k.contract_id = Est.contract_id
    ),
    k_i AS (
      SELECT
        *,
        COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent - total_landlord AS total_margin,
        (
          (COALESCE(total_rent, 0) + COALESCE(total_sc, 0)) - vat_sc - vat_rent - total_landlord
        ) /(
          CASE
            WHEN nights = 0 THEN 1
            ELSE nights
          END
        ) * 365 / 12 AS mg_booked,
        (
          (COALESCE(total_rent, 0) + COALESCE(total_sc, 0)) - vat_sc - vat_rent - total_landlord
        ) / (
          CASE
            WHEN (
              COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent
            ) = 0 THEN 1
            ELSE (
              COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent
            )
          END
        ) AS pct_mg
      FROM
        X
      WHERE
        true
      ORDER BY
        booking_id DESC
    )
  SELECT
    *
  FROM
    k_i
"""

spark.sql(margins_per_booking_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.margins_per_booking_7_days_deltas")

#### 2.2.1.3. Deltas Since Begining of Month
###### (Margins per booking) 

In [0]:
margins_per_booking_begining_month_deltas = """
with 
  payments AS (
      SELECT
        ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
        b.id AS booking_id,
        contracts.inserted_at :: DATE AS sell_date,
        -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        contracts.id as contract_id,
        CASE
          WHEN contracts.end_date > b.check_out THEN b.check_out --handle early departure check-out dates
          WHEN contracts.source = 'Hostify' THEN b.check_out --handle Hostify extensions check-out dates
          ELSE contracts.end_date
        END AS check_out,
        transactions.id as transaction_id,
        transaction_items.description,
        case
          when contract_lines.code = 'service_charge'
          and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price :: decimal / 100 :: decimal)
          else (transaction_items.price :: decimal / 100 :: decimal)
        end as payment
      from
        silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b
        left join silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft')
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND transactions.status not in ('canceled', 'expired', 'discarded')
        AND contracts.source NOT ILIKE '%extension%' --and transaction_items.price > 0
        and (transaction_items.description ILIKE '%rent%')
    ),
    ttl_rent AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        sell_date,
        contract_id,
        SUM(payment) AS total_rent
      FROM
        payments
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        contract_id
    ),
    comm AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN total_rent / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN total_rent / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN total_rent / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN total_rent / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN total_rent / 1.15 --otas
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN total_rent / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN total_rent / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN total_rent / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_rent / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Berlin', 'Paris') THEN total_rent / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN total_rent / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN total_rent / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN total_rent / 1.15
          ELSE total_rent
        END AS total_rent,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        sell_date,
        contract_id
      FROM
        ttl_rent
    ),
    remove_vat AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        --to avoid divisions by zero I'm adding a case when in the column of nights
        case
          when nights :: integer = 0 then 1
          else nights :: integer
        end as nights,
        source,
        sell_date,
        contract_id,
        case
          when city = 'Madrid'
          or city = 'Barcelona' then total_rent
          when city = 'Berlin' then total_rent
          else total_rent
        end as total_rent
      FROM
        comm
      WHERE
        true --AND check_out - check_in >= 365
        --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
      ORDER BY
        booking_id DESC
    ) ---------------------------------------------UNTIL HERE RENT----------------------------------------------------
  ,
    payments_sc AS (
      SELECT
        ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
        b.id AS booking_id,
        contracts.inserted_at :: DATE AS sell_date,
        -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        contracts.end_date as check_out,
        transactions.id as transaction_id,
        transaction_items.description,
        case
          when contract_lines.code = 'service_charge'
          and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price :: decimal / 100 :: decimal)
          else (transaction_items.price :: decimal / 100 :: decimal)
        end as payment
      from
        silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b
        left join silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft') -------------not including canceled, since no margin gets generated--------
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND transactions.status not in ('canceled', 'expired', 'discarded')
        AND contracts.source NOT ILIKE '%extension%' --and transaction_items.price > 0
        and (
          contract_lines.code = 'service_charge'
          or (
            transaction_items.description ILIKE '%service charge%'
          )
          or (
            transaction_items.description ILIKE '%service charge%'
          ) -----------------check this------------------ops discounts, refunds
          or (
            transaction_items.description ILIKE '%service_charge%'
          )
          or (
            transaction_items.description ILIKE '%administration fee%'
          ) ----------------check---------------including refunds, penalties
          or (
            transaction_items.description ILIKE '%admini fee%'
          )
        )
    ),
    ttl_rent_sc AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        SUM(payment) AS total_sc
      FROM
        payments_sc
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        state
    ),
    comm_sc AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN total_sc / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN total_sc / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN total_sc / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN total_sc / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN total_sc / 1.15 --otas
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN total_sc / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN total_sc / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN total_sc / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_sc / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Housinganywhere'
          AND city IN ('Berlin', 'Paris') THEN total_sc / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN total_sc / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN total_sc / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN total_sc / 1.15
          ELSE total_sc
        END AS total_sc,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date
      FROM
        ttl_rent_sc
    ),
    remove_vat_sc AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        case
          when city = 'Madrid'
          or city = 'Barcelona' then total_sc
          when city = 'Berlin' then total_sc
          else total_sc
        end as total_sc
      FROM
        comm_sc
      WHERE
        true --AND check_out - check_in >= 365
        --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
      ORDER BY
        booking_id DESC
    ) ---------------------------------------------UNTIL HERE SC----------------------------------------------------
  ,
    full_revenue AS (
      SELECT
        b.id AS booking_id,
        b.inserted_at :: DATE AS sell_date,
        b.source,
        b.state,
        a.codename,
        a.city,
        contracts.start_date as check_in,
        CASE
          WHEN contracts.end_date > b.check_out THEN b.check_out --handle early departure check-out dates
          WHEN contracts.source = 'Hostify' THEN b.check_out --handle Hostify extensions check-out dates
          ELSE contracts.end_date
        END AS check_out,
        CASE
          WHEN contract_lines.code = 'utilities'
          OR contract_lines.code = 'building_utilities' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS utilities,
        CASE
          WHEN contract_lines.code = 'final_cleaning' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS final_cleaning,
        CASE
          WHEN contract_lines.code = 'pets' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS pets,
        CASE
          WHEN contract_lines.code ILIKE '%extra_cleaning%' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS extra_cleaning,
        CASE
          WHEN contract_lines.code = 'airport_transport' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS airport_transport,
        CASE
          WHEN contract_lines.code = 'in_person_check_in' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS in_person_check_in,
        CASE
          WHEN contract_lines.code = 'parking' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS parking,
        CASE
          WHEN contract_lines.code = 'deposit' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS deposit,
        CASE
          WHEN contract_lines.code NOT IN (
            'rent',
            'service_charge',
            'utilities',
            'building_utilities',
            'final_cleaning',
            'pets',
            'airport_transport',
            'in_person_check_in',
            'parking',
            'deposit'
          )
          AND contract_lines.code NOT ILIKE '%extra_cleaning%' THEN transaction_items.price :: decimal / 100 :: decimal
        END AS other_revenue
      from
        silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b
        left join silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts on contracts.booking_id = b.id
        left join silver.backoffice.transactions TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
        left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) a on a.id = b.apartment_id
        left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
      WHERE
        true
        AND b.state NOT IN ('denied', 'retracted', 'draft')
        and contracts.status in ('booked', 'partial', 'agreed', 'canceled')
        AND contracts.source NOT ILIKE '%extension%'
        AND transactions.status not in ('canceled', 'expired', 'discarded')
    ),
    ttl_revenue AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        SUM(utilities) AS utilities,
        SUM(final_cleaning) AS final_cleaning,
        SUM(pets) AS pets,
        SUM(extra_cleaning) AS extra_cleaning,
        SUM(in_person_check_in) AS in_person_check_in,
        SUM(parking) AS parking,
        SUM(deposit) AS deposit,
        SUM(other_revenue) AS other_revenue
      FROM
        full_revenue
      GROUP BY
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        source,
        sell_date,
        state
    ),
    comm_rev AS (
      --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
      SELECT
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN utilities / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN utilities / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN utilities / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN utilities / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN utilities / 1.15 --otas (all except HA)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN utilities / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN utilities / 1.067
          WHEN sell_date >= '2025-01-01'
          AND source IN ('Homelike', 'HomeLike') THEN utilities / 1.08
          WHEN sell_date >= '2025-01-01'
          AND source = 'Spotahome' THEN utilities / 1.06
          WHEN sell_date >= '2025-01-01'
          AND source = 'Wunderflats' THEN utilities / 1.10
          WHEN sell_date >= '2025-01-01'
          AND source = 'Plum Guide' THEN utilities / 1.15
          ELSE utilities
        END AS utilities,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN final_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN final_cleaning / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN final_cleaning / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN final_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN final_cleaning / 1.15 --otas (only Airbnb)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN final_cleaning / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN final_cleaning / 1.067
          ELSE final_cleaning
        END AS final_cleaning,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN pets / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN pets / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN pets / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN pets / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN pets / 1.15 --otas (only Airbnb)
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT <= 90 THEN pets / 1.123
          WHEN sell_date >= '2025-01-01'
          AND source = 'Airbnb'
          AND nights :: INT > 90 THEN pets / 1.067
          ELSE pets
        END AS pets,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN extra_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN extra_cleaning / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN extra_cleaning / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN extra_cleaning / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN extra_cleaning / 1.15
          ELSE extra_cleaning
        END AS extra_cleaning,
        CASE
          --partners
          WHEN (
            booking_id IN (13226, 13508, 12787, 12601)
            OR (
              sell_date >= DATE('2024-09-02')
              AND source = 'Apartool'
            )
          ) THEN in_person_check_in / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT <= 90 THEN in_person_check_in / 1.09
          WHEN sell_date >= '2025-01-01'
          AND source = 'Altovita'
          AND nights :: INT > 90 THEN in_person_check_in / 1.05
          WHEN sell_date >= '2025-01-01'
          AND source = 'Silverdoor' THEN in_person_check_in / 1.15
          WHEN sell_date >= '2025-01-01'
          AND source = 'StaySitu' THEN in_person_check_in / 1.15
          ELSE in_person_check_in
        END AS in_person_check_in,
        other_revenue,
        parking,
        --not commissionable
        deposit,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date
      FROM
        ttl_revenue
    ),
    rev_wo_vat AS (
      SELECT
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        utilities,
        final_cleaning,
        pets,
        extra_cleaning,
        in_person_check_in,
        parking,
        deposit,
        other_revenue
      FROM
        comm_rev
      WHERE
        true
      ORDER BY
        booking_id DESC
    ) ------------------------------------UNTIL HERE FULL REVENUE-----------------------------------------------------------
  ,
    landlord_cogs_2 AS (
      SELECT
        SUM(landlord_daily_rent) :: decimal as landlord,
        trunc(day, 'month') as month,
        end_of_grace_period :: DATE + 1 as first_landlord_rent_payment_date,
        codename,
        city
      FROM
        silver.revenue.landlord TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) landlord
        LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON apartments.id = landlord.apartment_id
      GROUP BY
        month,
        codename,
        end_of_grace_period,
        city
    ),
    landlord_cogs AS (
      SELECT
        codename,
        city,
        month,
        landlord,
        CASE
          WHEN trunc(first_landlord_rent_payment_date, 'MM') = trunc(month, 'MM') THEN (
            trunc(month, 'MM') + INTERVAL '1' MONTH - INTERVAL '1' DAY - first_landlord_rent_payment_date :: DATE
          ) :: INT
          ELSE EXTRACT(
            DAY
            FROM
              trunc(month, 'mm') + INTERVAL '1' MONTH - INTERVAL '1' DAY
          ) :: INT
        END AS x,
        first_landlord_rent_payment_date
      FROM
        landlord_cogs_2
    ),
    split_month AS (
      SELECT
        codename,
        booking_id,
        city,
        check_in,
        check_out,
        explode(
          sequence(
            trunc(check_in :: date, 'month'),
            trunc(check_out :: date, 'month'),
            INTERVAL 1 month
          )
        ) AS month
      FROM
        rev_wo_vat
    ),
    ll_per_booking_2 AS (
      SELECT
        s.codename,
        s.city,
        s.booking_id,
        S.MONTH,
        CASE
          WHEN s.MONTH = trunc(check_in, 'month')
          AND s.MONTH = date_trunc('month', check_out) THEN (check_out :: DATE - check_in :: DATE) :: INT
          WHEN s.MONTH = trunc(check_in, 'month') THEN (
            trunc(check_in, 'month') + interval '1 month' - check_in :: DATE
          ) :: INT
          WHEN s.MONTH = date_trunc('month', check_out) THEN (
            check_out :: DATE - date_trunc('month', check_out) :: DATE
          ) :: INT
          ELSE (
            (trunc(s.MONTH, 'month') + interval '1 month') :: DATE - s.MONTH :: DATE
          ) :: INT
        END AS days
      FROM
        split_month s
    ),
    ll_per_booking AS (
      SELECT
        s.codename,
        s.city,
        s.booking_id,
        SUM(
          CASE
            WHEN trunc(first_landlord_rent_payment_date, 'month') = trunc(s.month, 'month')
            AND X <= days THEN X
            ELSE days
          END * (
            landlord :: DECIMAL / CASE
              WHEN X = 0 THEN 1
              ELSE X
            END
          )
        ) AS Landlord
      FROM
        ll_per_booking_2 s
        LEFT JOIN landlord_cogs l ON s.MONTH = l.MONTH
        AND s.codename = l.codename
      GROUP BY
        s.codename,
        s.city,
        s.booking_id
      ORDER BY
        s.booking_id DESC
    ) --------------------------------------UNTIL HERE LANDLORD, CONSIDERING GRD AND CARENCIA-----------------
  ,
    final_2 AS (
      SELECT
        remove_vat.*,
        rev_wo_vat.utilities,
        rev_wo_vat.final_cleaning,
        rev_wo_vat.pets,
        rev_wo_vat.extra_cleaning,
        rev_wo_vat.in_person_check_in,
        rev_wo_vat.parking,
        rev_wo_vat.deposit,
        rev_wo_vat.other_revenue,
        remove_vat_sc.total_sc,
        ll_per_booking.Landlord AS total_landlord
      FROM
        remove_vat
        LEFT JOIN rev_wo_vat ON rev_wo_vat.booking_id = remove_vat.booking_id
        LEFT JOIN remove_vat_sc ON rev_wo_vat.booking_id = remove_vat_sc.booking_id
        LEFT JOIN ll_per_booking ON rev_wo_vat.booking_id = ll_per_booking.booking_id
    ),
    final AS (
      SELECT
        *,
        COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0) AS total_rev,
        total_landlord / nights *(365 / 12) AS avg_monthly_landlord,
        case
          ---VAT of total_revenue
          when city = 'Madrid'
          or city = 'Barcelona' then (
            COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.1 * 0.1
          when city = 'Berlin' then (
            COALESCE(total_rent, 0) + COALESCE(total_sc, 0) + COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.07 * 0.07
          else 0
        end as vat_revenue,
        case
          ---VAT of total_rent
          when city = 'Madrid'
          or city = 'Barcelona' then total_rent / 1.1 * 0.1
          when city = 'Berlin' then total_rent / 1.07 * 0.07
          else 0
        end as vat_rent,
        case
          ---VAT of total_sc
          when city = 'Madrid'
          or city = 'Barcelona' then total_sc / 1.1 * 0.1
          when city = 'Berlin' then total_sc / 1.07 * 0.07
          else 0
        end as vat_sc,
        case
          ---VAT of other revenue concepts
          when city = 'Madrid'
          or city = 'Barcelona' then (
            COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.1 * 0.1
          when city = 'Berlin' then (
            COALESCE(utilities, 0) + COALESCE(final_cleaning, 0) + COALESCE(pets, 0) + COALESCE(extra_cleaning, 0) + COALESCE(in_person_check_in, 0) + COALESCE(parking, 0) + COALESCE(other_revenue, 0)
          ) / 1.07 * 0.07
          else 0
        end as vat_other_rev
      FROM
        final_2
    ),
    Est AS (
      SELECT
        k.contract_id,
        cl.code,
        (cl.price :: DECIMAL / 100) AS SC_Price
      FROM
        final k
        LEFT JOIN silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) cl ON k.contract_id = cl.contract_id
      WHERE
        code = 'service_charge'
        AND k.city = 'Paris'
        AND k.sell_date >= DATE('2023-12-04')
    ),
    X AS (
      SELECT
        k.codename,
        k.booking_id,
        k.city,
        k.check_in,
        k.check_out,
        k.source,
        k.sell_date,
        k.contract_id,
        k.nights,
        k.total_rent,
        CASE
          WHEN (
            city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
            AND k.total_sc IS NULL
          ) THEN utilities - (
            CASE
              WHEN SC_Price IS NULL THEN 0
              ELSE SC_Price
            END
          )
          ELSE k.utilities
        END AS utilities,
        CASE
          WHEN (
            city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
            AND k.total_sc IS NULL
          ) THEN (
            CASE
              WHEN SC_Price IS NULL THEN 0
              ELSE SC_Price
            END
          )
          ELSE k.total_sc
        END AS total_sc,
        k.final_cleaning,
        k.pets,
        k.extra_cleaning,
        k.in_person_check_in,
        k.parking,
        k.deposit,
        k.other_revenue,
        k.total_landlord,
        k.total_rev,
        k.avg_monthly_landlord,
        k.vat_revenue,
        k.vat_rent,
        k.vat_sc,
        k.vat_other_rev
      FROM
        final k
        LEFT JOIN Est ON k.contract_id = Est.contract_id
    ),
    k_i AS (
      SELECT
        *,
        COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent - total_landlord AS total_margin,
        (
          (COALESCE(total_rent, 0) + COALESCE(total_sc, 0)) - vat_sc - vat_rent - total_landlord
        ) /(
          CASE
            WHEN nights = 0 THEN 1
            ELSE nights
          END
        ) * 365 / 12 AS mg_booked,
        (
          (COALESCE(total_rent, 0) + COALESCE(total_sc, 0)) - vat_sc - vat_rent - total_landlord
        ) / (
          CASE
            WHEN (
              COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent
            ) = 0 THEN 1
            ELSE (
              COALESCE(total_rent, 0) + COALESCE(total_sc, 0) - vat_sc - vat_rent
            )
          END
        ) AS pct_mg
      FROM
        X
      WHERE
        true
      ORDER BY
        booking_id DESC
    )
  SELECT
    *
  FROM
    k_i
"""

spark.sql(margins_per_booking_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.margins_per_booking_begining_month_deltas")

### 2.2.2. Bookings Maser

#### 2.2.2.1. Actuals 
###### (Bookings Master) 

In [0]:
bookings_masters = """
with 
    contracts_1 as (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY booking_id ORDER BY inserted_at ASC) AS RN
        FROM silver.backoffice.contracts
        WHERE true AND status IN ('booked', 'partial')
    )

    ,CTE AS (
        SELECT 
            bookings.id, 
            contracts_1.start_date::DATE as check_in, 
            contracts_1.inserted_at,
            CASE 
                WHEN contracts_1.end_date > bookings.check_out THEN bookings.check_out::DATE --handle early departure check-out dates
                WHEN contracts_1.source = 'Hostify' THEN bookings.check_out::DATE --handle Hostify extensions check-out dates
                ELSE contracts_1.end_date::DATE
            END AS check_out,
            contracts_1.id as c_id,
            CASE 
                WHEN contracts_1.end_date > bookings.check_out THEN 'ED'
                ELSE 'Original booking'
            END AS booking_type_,
            codename, 
            apartments.city,
            stay_reason, bookings.state, profiles.first_name AS name, profiles.last_name, profiles.email,bookings.source as bookings_source, contracts_1.source as contracts_source, bookings.profile_guest_id as guest_id
        
        FROM silver.backoffice.bookings
            LEFT JOIN silver.backoffice.apartments on backoffice.apartments.id = backoffice.bookings.apartment_id
            LEFT JOIN silver.backoffice.profiles ON backoffice.bookings.profile_guest_id = backoffice.profiles.id
            LEFT JOIN contracts_1 on backoffice.bookings.id = contracts_1.booking_id

        WHERE true
            AND bookings.state NOT IN ('canceled', 'denied', 'retracted', 'draft')
            AND bookings.source != 'Direct Booking'
            AND bookings.apartment_id != 673 -- TEST PROPERTY
            AND (profiles.id IN (10820, 12078, 11963, 11986) 
                OR (profiles.email != 'guests@ukio.com' OR profiles.email IS NULL))
            AND contracts_1.rn = 1

        group by 
            bookings.id,bookings.profile_guest_id,interior_sqm,contracts_1.inserted_at,nights,codename, apartments.city,
            stay_reason, profiles.first_name, profiles.last_name,profiles.email,bookings.source, bookings.state,profiles.inserted_at, 
            contracts_1.start_date, contracts_1.end_date, contracts_1.source, bookings.check_out, contracts_1.id
            
        ORDER BY id
    )

    ,mg_booked AS (
        SELECT * FROM silver.finance.margins_per_booking
    )

    ,final_2 AS (
        SELECT 
            cte.*, 
            cte.check_out - cte.check_in as nights,
            mg_booked.mg_booked::DECIMAL, 
            mg_booked.total_margin::DECIMAL,
            mg_booked.total_rent::DECIMAL,
            mg_booked.total_rev::DECIMAL,
            mg_booked.total_sc::DECIMAL,   
            mg_booked.utilities::DECIMAL,
            mg_booked.final_cleaning::DECIMAL,
            mg_booked.extra_cleaning::DECIMAL,
            mg_booked.pets::DECIMAL,
            mg_booked.in_person_check_in,
            mg_booked.parking,
            mg_booked.deposit::DECIMAL,
            mg_booked.other_revenue::DECIMAL,
            mg_booked.avg_monthly_landlord::DECIMAL,
            mg_booked.total_landlord::DECIMAL,
            mg_booked.vat_revenue::DECIMAL,
            mg_booked.vat_rent::DECIMAL,
            mg_booked.vat_sc::DECIMAL,
            mg_booked.vat_other_rev::DECIMAL,
            cte.c_id as contract_id
        
        FROM cte
            LEFT JOIN mg_booked ON cte.id::INT = mg_booked.booking_id::INT
        
        WHERE name NOT ILIKE '%test%'

        ORDER BY inserted_at DESC
    )

    ,test as (
        SELECT 
            --final_2.*, 
            final_2.id, check_in, final_2.inserted_at, check_out, final_2.codename, final_2.city, stay_reason, state, final_2.name, last_name, email, bookings_source, contracts_source, 
            guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, parking, other_revenue, vat_revenue,
            vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit 
        
        FROM final_2 
            LEFT JOIN silver.backoffice.apartments on final_2.codename = apartments.codename and final_2.city = apartments.city 
    )

    ,A AS (
        SELECT * from silver.finance.overlapping
    )

    ,B AS (
        SELECT 
        test.*, MIN(a.ci) ci, MIN(a.co) co, a.inserted_at as inserted_at_2 
        
        FROM test 
        LEFT JOIN A ON a.CONTRACT_ID_FILLED = test.contract_id

        GROUP BY 
        test.id, test.check_in, test.inserted_at, test.check_out, test.codename, test.city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning,
        pets, in_person_check_in, parking, other_revenue, vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit, a.inserted_at
    )

    ,C AS (
        SELECT 
            id, COALESCE(ci, check_in) check_in, 
            CASE 
                WHEN inserted_at_2 > check_in THEN check_in 
                WHEN inserted_at_2 IS NULL THEN inserted_at
            ELSE inserted_at_2 END AS inserted_at,
            COALESCE(co,check_out) check_out, codename, city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning,
            pets, in_person_check_in, parking, other_revenue, vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit
        
        FROM B
    )

    SELECT id, check_in, check_out, inserted_at, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, other_revenue,
    vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, 
    deposit FROM C WHERE inserted_at >= DATE('2025-02-01') 
    UNION ALL 
    SELECT id, check_in, check_out, inserted_at, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, other_revenue,
    vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, 
    deposit FROM silver.finance.materialized_bookings_masters
"""

spark.sql(bookings_masters).write.mode("overwrite").saveAsTable("silver.finance.bookings_masters")

#### 2.2.2.2. Deltas 7 Days Ago
###### (Bookings Master)

In [0]:
bookings_masters_7_days_deltas = """
with 
    contracts_1 as (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY booking_id ORDER BY inserted_at ASC) AS RN
        FROM silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts
        WHERE true AND status IN ('booked', 'partial')
    )

    ,CTE AS (
        SELECT 
            bookings.id, 
            contracts_1.start_date::DATE as check_in, 
            contracts_1.inserted_at,
            CASE 
                WHEN contracts_1.end_date > bookings.check_out THEN bookings.check_out::DATE --handle early departure check-out dates
                WHEN contracts_1.source = 'Hostify' THEN bookings.check_out::DATE --handle Hostify extensions check-out dates
                ELSE contracts_1.end_date::DATE
            END AS check_out,
            contracts_1.id as c_id,
            CASE 
                WHEN contracts_1.end_date > bookings.check_out THEN 'ED'
                ELSE 'Original booking'
            END AS booking_type_,
            codename, 
            apartments.city,
            stay_reason, bookings.state, profiles.first_name AS name, profiles.last_name, profiles.email,bookings.source as bookings_source, contracts_1.source as contracts_source, bookings.profile_guest_id as guest_id
        
        FROM silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments on apartments.id = bookings.apartment_id
            LEFT JOIN silver.backoffice.profiles TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) profiles ON bookings.profile_guest_id = profiles.id
            LEFT JOIN contracts_1 on bookings.id = contracts_1.booking_id

        WHERE true
            AND bookings.state NOT IN ('canceled', 'denied', 'retracted', 'draft')
            AND bookings.source != 'Direct Booking'
            AND bookings.apartment_id != 673 -- TEST PROPERTY
            AND (profiles.id IN (10820, 12078, 11963, 11986) 
                OR (profiles.email != 'guests@ukio.com' OR profiles.email IS NULL))
            AND contracts_1.rn = 1

        group by 
            bookings.id,bookings.profile_guest_id,interior_sqm,contracts_1.inserted_at,nights,codename, apartments.city,
            stay_reason, profiles.first_name, profiles.last_name,profiles.email,bookings.source, bookings.state,profiles.inserted_at, 
            contracts_1.start_date, contracts_1.end_date, contracts_1.source, bookings.check_out, contracts_1.id
            
        ORDER BY id
    )

    ,mg_booked AS (
        SELECT * FROM silver.finance.margins_per_booking_7_days_deltas
    )

    ,final_2 AS (
        SELECT 
            cte.*, 
            cte.check_out - cte.check_in as nights,
            mg_booked.mg_booked::DECIMAL, 
            mg_booked.total_margin::DECIMAL,
            mg_booked.total_rent::DECIMAL,
            mg_booked.total_rev::DECIMAL,
            mg_booked.total_sc::DECIMAL,   
            mg_booked.utilities::DECIMAL,
            mg_booked.final_cleaning::DECIMAL,
            mg_booked.extra_cleaning::DECIMAL,
            mg_booked.pets::DECIMAL,
            mg_booked.in_person_check_in,
            mg_booked.parking,
            mg_booked.deposit::DECIMAL,
            mg_booked.other_revenue::DECIMAL,
            mg_booked.avg_monthly_landlord::DECIMAL,
            mg_booked.total_landlord::DECIMAL,
            mg_booked.vat_revenue::DECIMAL,
            mg_booked.vat_rent::DECIMAL,
            mg_booked.vat_sc::DECIMAL,
            mg_booked.vat_other_rev::DECIMAL,
            cte.c_id as contract_id
        
        FROM cte
            LEFT JOIN mg_booked ON cte.id::INT = mg_booked.booking_id::INT
        
        WHERE name NOT ILIKE '%test%'

        ORDER BY inserted_at DESC
    )

    ,test as (
        SELECT 
            --final_2.*, 
            final_2.id, check_in, final_2.inserted_at, check_out, final_2.codename, final_2.city, stay_reason, state, final_2.name, last_name, email, bookings_source, contracts_source, 
            guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, parking, other_revenue, vat_revenue,
            vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit 
        
        FROM final_2 
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments on final_2.codename = apartments.codename and final_2.city = apartments.city 
    )

    ,A AS (
        SELECT * from silver.finance.overlapping
    )

    ,B AS (
        SELECT 
        test.*, MIN(a.ci) ci, MIN(a.co) co, a.inserted_at as inserted_at_2 
        
        FROM test 
        LEFT JOIN A ON a.CONTRACT_ID_FILLED = test.contract_id

        GROUP BY 
        test.id, test.check_in, test.inserted_at, test.check_out, test.codename, test.city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning,
        pets, in_person_check_in, parking, other_revenue, vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit, a.inserted_at
    )

    ,C AS (
        SELECT 
            id, COALESCE(ci, check_in) check_in, 
            CASE 
                WHEN inserted_at_2 > check_in THEN check_in 
                WHEN inserted_at_2 IS NULL THEN inserted_at
            ELSE inserted_at_2 END AS inserted_at,
            COALESCE(co,check_out) check_out, codename, city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning,
            pets, in_person_check_in, parking, other_revenue, vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit
        
        FROM B
    )

    SELECT id, check_in, check_out, inserted_at, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, other_revenue,
    vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, 
    deposit FROM C
"""

spark.sql(bookings_masters_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.bookings_masters_7_days_deltas")

#### 2.2.2.3. Deltas Since Begining of Month
###### (Bookings Master)

In [0]:
bookings_masters_begining_month_deltas = """
with 
    contracts_1 as (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY booking_id ORDER BY inserted_at ASC) AS RN
        FROM silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts
        WHERE true AND status IN ('booked', 'partial')
    )

    ,CTE AS (
        SELECT 
            bookings.id, 
            contracts_1.start_date::DATE as check_in, 
            contracts_1.inserted_at,
            CASE 
                WHEN contracts_1.end_date > bookings.check_out THEN bookings.check_out::DATE --handle early departure check-out dates
                WHEN contracts_1.source = 'Hostify' THEN bookings.check_out::DATE --handle Hostify extensions check-out dates
                ELSE contracts_1.end_date::DATE
            END AS check_out,
            contracts_1.id as c_id,
            CASE 
                WHEN contracts_1.end_date > bookings.check_out THEN 'ED'
                ELSE 'Original booking'
            END AS booking_type_,
            codename, 
            apartments.city,
            stay_reason, bookings.state, profiles.first_name AS name, profiles.last_name, profiles.email,bookings.source as bookings_source, contracts_1.source as contracts_source, bookings.profile_guest_id as guest_id
        
        FROM silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments on apartments.id = bookings.apartment_id
            LEFT JOIN silver.backoffice.profiles TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) profiles ON bookings.profile_guest_id = profiles.id
            LEFT JOIN contracts_1 on bookings.id = contracts_1.booking_id

        WHERE true
            AND bookings.state NOT IN ('canceled', 'denied', 'retracted', 'draft')
            AND bookings.source != 'Direct Booking'
            AND bookings.apartment_id != 673 -- TEST PROPERTY
            AND (profiles.id IN (10820, 12078, 11963, 11986) 
                OR (profiles.email != 'guests@ukio.com' OR profiles.email IS NULL))
            AND contracts_1.rn = 1

        group by 
            bookings.id,bookings.profile_guest_id,interior_sqm,contracts_1.inserted_at,nights,codename, apartments.city,
            stay_reason, profiles.first_name, profiles.last_name,profiles.email,bookings.source, bookings.state,profiles.inserted_at, 
            contracts_1.start_date, contracts_1.end_date, contracts_1.source, bookings.check_out, contracts_1.id
            
        ORDER BY id
    )

    ,mg_booked AS (
        SELECT * FROM silver.finance.margins_per_booking_begining_month_deltas
    )

    ,final_2 AS (
        SELECT 
            cte.*, 
            cte.check_out - cte.check_in as nights,
            mg_booked.mg_booked::DECIMAL, 
            mg_booked.total_margin::DECIMAL,
            mg_booked.total_rent::DECIMAL,
            mg_booked.total_rev::DECIMAL,
            mg_booked.total_sc::DECIMAL,   
            mg_booked.utilities::DECIMAL,
            mg_booked.final_cleaning::DECIMAL,
            mg_booked.extra_cleaning::DECIMAL,
            mg_booked.pets::DECIMAL,
            mg_booked.in_person_check_in,
            mg_booked.parking,
            mg_booked.deposit::DECIMAL,
            mg_booked.other_revenue::DECIMAL,
            mg_booked.avg_monthly_landlord::DECIMAL,
            mg_booked.total_landlord::DECIMAL,
            mg_booked.vat_revenue::DECIMAL,
            mg_booked.vat_rent::DECIMAL,
            mg_booked.vat_sc::DECIMAL,
            mg_booked.vat_other_rev::DECIMAL,
            cte.c_id as contract_id
        
        FROM cte
            LEFT JOIN mg_booked ON cte.id::INT = mg_booked.booking_id::INT
        
        WHERE name NOT ILIKE '%test%'

        ORDER BY inserted_at DESC
    )

    ,test as (
        SELECT 
            --final_2.*, 
            final_2.id, check_in, final_2.inserted_at, check_out, final_2.codename, final_2.city, stay_reason, state, final_2.name, last_name, email, bookings_source, contracts_source, 
            guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, parking, other_revenue, vat_revenue,
            vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit 
        
        FROM final_2 
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments on final_2.codename = apartments.codename and final_2.city = apartments.city 
    )

    ,A AS (
        SELECT * from silver.finance.overlapping
    )

    ,B AS (
        SELECT 
        test.*, MIN(a.ci) ci, MIN(a.co) co, a.inserted_at as inserted_at_2 
        
        FROM test 
        LEFT JOIN A ON a.CONTRACT_ID_FILLED = test.contract_id

        GROUP BY 
        test.id, test.check_in, test.inserted_at, test.check_out, test.codename, test.city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning,
        pets, in_person_check_in, parking, other_revenue, vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit, a.inserted_at
    )

    ,C AS (
        SELECT 
            id, COALESCE(ci, check_in) check_in, 
            CASE 
                WHEN inserted_at_2 > check_in THEN check_in 
                WHEN inserted_at_2 IS NULL THEN inserted_at
            ELSE inserted_at_2 END AS inserted_at,
            COALESCE(co,check_out) check_out, codename, city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities, final_cleaning, extra_cleaning,
            pets, in_person_check_in, parking, other_revenue, vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, deposit
        
        FROM B
    )

    SELECT id, check_in, check_out, inserted_at, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, final_cleaning, extra_cleaning, pets, in_person_check_in, other_revenue,
    vat_revenue, vat_rent, vat_sc, vat_other_rev, avg_monthly_landlord, total_landlord, mg_booked, total_margin, 
    deposit FROM C
"""

spark.sql(bookings_masters_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.bookings_masters_begining_month_deltas")

### 2.2.3. Cancelled Bookings

In [0]:
canceled_bookings_master = """
with contracts_1 as (
      SELECT *, ROW_NUMBER() OVER (PARTITION BY booking_id ORDER BY inserted_at ASC) AS RN
      FROM silver.backoffice.contracts
      WHERE status = 'canceled'
    ),
    cte AS (
      SELECT
        bookings.id,
        contracts_1.start_date :: DATE as check_in,
        bookings.inserted_at,
        COALESCE(contracts_1.canceled_at, contracts_1.updated_at) canceled_at,
        CASE
          WHEN contracts_1.end_date > bookings.check_out THEN bookings.check_out :: DATE --handle early departure check-out dates
          WHEN contracts_1.source = 'Hostify' THEN bookings.check_out :: DATE --handle Hostify extensions check-out dates
          ELSE contracts_1.end_date :: DATE
        END AS check_out,
        CASE
          WHEN contracts_1.end_date > bookings.check_out THEN 'ED'
          ELSE 'Original booking'
        END AS booking_type_,
        codename, apartments.city,stay_reason,bookings.state,profiles.first_name AS name,profiles.last_name,profiles.email,
        bookings.source as bookings_source,contracts_1.source as contracts_source,bookings.profile_guest_id as guest_id
      FROM
        silver.backoffice.bookings
        LEFT JOIN silver.backoffice.apartments on backoffice.apartments.id = backoffice.bookings.apartment_id
        LEFT JOIN silver.backoffice.profiles ON backoffice.bookings.profile_guest_id = backoffice.profiles.id
        LEFT JOIN contracts_1 on backoffice.bookings.id = contracts_1.booking_id
      WHERE
        true
        AND bookings.state IN ('canceled')
        AND bookings.source != 'Direct Booking'
        AND bookings.apartment_id != 673 -- TEST PROPERTY
        AND (
          profiles.id IN (10820, 12078, 11963, 11986)
          OR (
            profiles.email != 'guests@ukio.com'
            OR profiles.email IS NULL
          )
        )
        AND contracts_1.rn = 1
      group by
        bookings.id,bookings.profile_guest_id,interior_sqm,bookings.inserted_at,nights,codename,apartments.city,stay_reason,profiles.first_name,profiles.last_name,
        profiles.email,bookings.source,bookings.state,profiles.inserted_at,contracts_1.start_date,contracts_1.end_date,contracts_1.source,contracts_1.canceled_at,
        bookings.check_out,contracts_1.updated_at
      ORDER BY
        id
    ),
    mg_booked AS (
      SELECT
        *
      FROM
        silver.finance.materialized_margins_per_booking
    ),
    final_2 AS (
      SELECT
        cte.*,cte.check_out - cte.check_in as nights,mg_booked.mg_booked :: DECIMAL,mg_booked.total_margin :: DECIMAL,mg_booked.total_rent :: DECIMAL,
        mg_booked.total_rev :: DECIMAL,mg_booked.total_sc :: DECIMAL,mg_booked.utilities :: DECIMAL,mg_booked.final_cleaning :: DECIMAL, mg_booked.extra_cleaning :: DECIMAL,
        mg_booked.pets :: DECIMAL,mg_booked.in_person_check_in,mg_booked.parking,mg_booked.deposit :: DECIMAL,mg_booked.other_revenue :: DECIMAL,mg_booked.avg_monthly_landlord :: DECIMAL,
        mg_booked.total_landlord :: DECIMAL,mg_booked.vat_revenue :: DECIMAL,mg_booked.vat_rent :: DECIMAL,mg_booked.vat_sc :: DECIMAL, mg_booked.vat_other_rev :: DECIMAL,
        mg_booked.contract_id
      FROM
        cte
        LEFT JOIN mg_booked ON cte.id :: INT = mg_booked.booking_id :: INT
      WHERE
        name NOT ILIKE '%test%'
      ORDER BY
        inserted_at DESC
    ),
    test as (
      SELECT
        --final_2.*,
        final_2.id,check_in,final_2.inserted_at,check_out,final_2.codename,final_2.city,stay_reason,state,final_2.name,last_name,
        email,bookings_source,contracts_source,guest_id,contract_id,nights,total_rev,total_rent,total_sc,utilities,final_cleaning,
        extra_cleaning,pets,in_person_check_in,parking,other_revenue,vat_revenue,      vat_rent,vat_sc,vat_other_rev,avg_monthly_landlord,total_landlord,mg_booked,total_margin,
        deposit,final_2.canceled_at
      FROM
        final_2
        LEFT JOIN silver.backoffice.apartments on final_2.codename = apartments.codename
        and final_2.city = apartments.city
      WHERE
        true
    )
  SELECT
    id,check_in,check_out,inserted_at,canceled_at,codename,city,stay_reason,state,name,last_name,email,bookings_source,contracts_source,guest_id,
    contract_id,nights,total_rev,total_rent,total_sc,utilities,final_cleaning,extra_cleaning,pets,in_person_check_in,parking,other_revenue,vat_revenue,
    vat_rent,vat_sc,vat_other_rev,avg_monthly_landlord,total_landlord,mg_booked,total_margin,deposit
  FROM
    test
  WHERE
    inserted_at >= DATE('2025-01-01')
  UNION ALL
  SELECT
    id,check_in,check_out,inserted_at,canceled_at,codename,city,stay_reason,state,name,last_name,email,bookings_source,contracts_source,guest_id,
    contract_id,nights,total_rev,total_rent,total_sc,utilities,final_cleaning,extra_cleaning,pets,in_person_check_in,parking,other_revenue,vat_revenue,
    vat_rent,vat_sc,vat_other_rev,avg_monthly_landlord,total_landlord,mg_booked,total_margin,deposit
  FROM
    silver.finance.materialized_canceled_bookings_master
  ORDER BY
    canceled_at DESC
"""

spark.sql(canceled_bookings_master).write.mode("overwrite").saveAsTable("silver.finance.canceled_bookings_master")

## 2.3. Extensions

### 2.3.1. Margins per extension

#### 2.3.1.1. Actuals
###### (Margins per Extension)

In [0]:
margins_per_extension = """
with 
    payments AS ( 
        SELECT ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
            b.id AS booking_id,
            contracts.inserted_at::DATE AS sell_date, -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.start_date as check_in,
            contracts.id as contract_id,
            contracts.end_date as check_out,
            transactions.id as transaction_id, transaction_items.description,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price::decimal/100::decimal) 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment
        
        from silver.backoffice.bookings b
            left join silver.backoffice.contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions  on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft')
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and (transaction_items.description ILIKE '%rent%')
    )

    ,ttl_rent AS (
    SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        sell_date,
        contract_id,
        SUM(payment) AS total_rent
        FROM payments
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, contract_id
    )

    ,comm AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN total_rent/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN total_rent/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN total_rent/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN total_rent/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN total_rent/1.15
            
            --otas
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN total_rent/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN total_rent/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN total_rent/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_rent/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Berlin', 'Paris') THEN total_rent/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN total_rent/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN total_rent/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN total_rent/1.15
            
            ELSE total_rent
        END AS total_rent, booking_id,city,codename,check_in,check_out,nights,source,sell_date,
        contract_id
        FROM ttl_rent
    )

    ,remove_vat AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        --to avoid divisions by zero I'm adding a case when in the column of nights
        case when nights::integer = 0 then 1 else nights::integer end as nights,
        source,
        sell_date,
        contract_id,
        case 
            when city = 'Madrid' or city = 'Barcelona' then total_rent 
            when city = 'Berlin' then total_rent
            else total_rent 
        end as total_rent
        FROM comm
            WHERE true 
            --AND check_out - check_in >= 365
            --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
        ORDER BY booking_id DESC
    ) 
    ---------------------------------------------UNTIL HERE RENT----------------------------------------------------
    ,payments_sc AS ( 
        SELECT ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
            b.id AS booking_id,
            contracts.inserted_at::DATE AS sell_date, -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.id as contract_id,
            contracts.start_date as check_in,
            contracts.end_date as check_out, transactions.id as transaction_id, transaction_items.description,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price::decimal/100::decimal) 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment
        
        from silver.backoffice.bookings b
            left join silver.backoffice.contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions  on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft') -------------not including canceled, since no margin gets generated--------
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and (contract_lines.code = 'service_charge'
                or (transaction_items.description ILIKE '%service charge%')
                or (transaction_items.description ILIKE '%service charge%') -----------------check this------------------ops discounts, refunds
                or (transaction_items.description ILIKE '%service_charge%')
                or (transaction_items.description ILIKE '%administration fee%') ----------------check---------------including refunds, penalties
                or (transaction_items.description ILIKE '%admini fee%')
                )
    )

    ,ttl_rent_sc AS (
        SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_in - check_out AS nights,
        source,
        contract_id,
        state,
        sell_date,
        SUM(payment) AS total_sc
        FROM payments_sc
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, state, contract_id
    )

    ,comm_sc AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN total_sc/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN total_sc/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN total_sc/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN total_sc/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN total_sc/1.15
            
            --otas
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN total_sc/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN total_sc/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN total_sc/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_sc/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Berlin', 'Paris') THEN total_sc/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN total_sc/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN total_sc/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN total_sc/1.15
            
            ELSE total_sc
        END AS total_sc,
        booking_id,
        city,
        codename,
        check_in,
        check_out, contract_id,
        check_out - check_in AS nights,
        source,
        state,
        sell_date
        FROM ttl_rent_sc
    )

    ,remove_vat_sc AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source, contract_id,
        state,
        sell_date,
        case 
            when city = 'Madrid' or city = 'Barcelona' then total_sc 
            when city = 'Berlin' then total_sc
            else total_sc 
        end as total_sc
        FROM comm_sc
            WHERE true 
            --AND check_out - check_in >= 365
            --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
        ORDER BY booking_id DESC
    ) 
    ---------------------------------------------UNTIL HERE SC----------------------------------------------------

    ,full_revenue AS ( 
        SELECT 
            b.id AS booking_id,
            b.inserted_at::DATE AS sell_date, 
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.id as contract_id,
            contracts.start_date as check_in,
            contracts.end_date as check_out,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= '10-27-2023' then (transaction_items.price::decimal/100::decimal)/1.02 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment,
            CASE 
                WHEN contract_lines.code = 'utilities' OR contract_lines.code = 'building_utilities' THEN transaction_items.price::decimal/100::decimal
            END AS utilities,
            CASE 
                WHEN contract_lines.code = 'final_cleaning' THEN transaction_items.price::decimal/100::decimal
            END AS final_cleaning,
            CASE 
                WHEN contract_lines.code = 'pets' THEN transaction_items.price::decimal/100::decimal
            END AS pets,
            CASE 
                WHEN contract_lines.code ILIKE '%extra_cleaning%' THEN transaction_items.price::decimal/100::decimal
            END AS extra_cleaning,
            CASE 
                WHEN contract_lines.code = 'airport_transport' THEN transaction_items.price::decimal/100::decimal
            END AS airport_transport,
            CASE 
                WHEN contract_lines.code = 'in_person_check_in' THEN transaction_items.price::decimal/100::decimal
            END AS in_person_check_in,
            CASE 
                WHEN contract_lines.code = 'parking' THEN transaction_items.price::decimal/100::decimal
            END AS parking,
            CASE 
                WHEN contract_lines.code = 'deposit' THEN transaction_items.price::decimal/100::decimal
            END AS deposit,
            
            CASE 
                WHEN contract_lines.code NOT IN ('rent', 'service_charge', 'utilities', 'building_utilities', 'final_cleaning', 'pets', 'airport_transport', 'in_person_check_in', 'parking', 'deposit') AND contract_lines.code NOT ILIKE '%extra_cleaning%' 
                    THEN transaction_items.price::decimal/100::decimal
            END AS other_revenue
        
        from silver.backoffice.bookings b
            left join silver.backoffice.contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions  on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft')
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and contract_lines.code <> 'deposit'
    )

    ,ttl_revenue AS (
        SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        contract_id,
        source,
        state,
        sell_date,
        SUM(utilities) AS utilities,
        SUM(final_cleaning) AS final_cleaning,
        SUM(pets) AS pets,
        SUM(extra_cleaning) AS extra_cleaning,
        SUM(in_person_check_in) AS in_person_check_in,
        SUM(parking) AS parking,
        SUM(deposit) AS deposit,
        SUM(other_revenue) AS other_revenue
        FROM full_revenue
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, contract_id, state
    )

    ,comm_rev AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN utilities/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN utilities/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN utilities/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN utilities/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN utilities/1.15
            
            --otas (all except HA)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN utilities/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN utilities/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN utilities/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN utilities/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN utilities/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN utilities/1.15
            
            ELSE utilities
        END AS utilities,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN final_cleaning/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN final_cleaning/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN final_cleaning/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN final_cleaning/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN final_cleaning/1.15
            
            --otas (only Airbnb)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN final_cleaning/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN final_cleaning/1.067
            
            ELSE final_cleaning
        END AS final_cleaning,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN pets/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN pets/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN pets/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN pets/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN pets/1.15
            
            --otas (only Airbnb)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN pets/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN pets/1.067
            
            ELSE pets
        END AS pets,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN extra_cleaning/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN extra_cleaning/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN extra_cleaning/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN extra_cleaning/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN extra_cleaning/1.15
            
            ELSE extra_cleaning
        END AS extra_cleaning,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN in_person_check_in/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN in_person_check_in/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN in_person_check_in/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN in_person_check_in/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN in_person_check_in/1.15
            
            ELSE in_person_check_in
        END AS in_person_check_in,
        
        other_revenue,
        parking, --not commissionable
        deposit,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        contract_id
        FROM ttl_revenue
    )

    ,rev_wo_vat AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        utilities, 
        contract_id,
        final_cleaning,
        pets,
        extra_cleaning,
        in_person_check_in,
        parking,
        deposit,
        other_revenue
        
        FROM comm_rev
            WHERE true 
        ORDER BY booking_id DESC
    )

    ------------------------------------UNTIL HERE FULL REVENUE-----------------------------------------------------------

    ,landlord_cogs_2 AS (
        SELECT 
            SUM(landlord_daily_rent)::decimal as landlord, 
            trunc(day, 'month') as month, end_of_grace_period::DATE + 1 as first_landlord_rent_payment_date,
            codename, city

        FROM silver.revenue.landlord
            LEFT JOIN silver.Backoffice.apartments ON Backoffice.apartments.id = revenue.landlord.apartment_id
        
        GROUP BY month, codename, end_of_grace_period, city
    )

    ,landlord_cogs AS (
        SELECT
            codename, city, month, landlord,
            CASE 
                WHEN trunc(first_landlord_rent_payment_date, 'MM') = trunc(month, 'MM') THEN (trunc(month, 'MM') + INTERVAL '1' MONTH - INTERVAL '1' DAY - first_landlord_rent_payment_date::DATE)::INT
                ELSE EXTRACT(DAY FROM trunc(month, 'mm') + INTERVAL '1' MONTH - INTERVAL '1' DAY)::INT
            END AS x, 
            first_landlord_rent_payment_date
        
        FROM landlord_cogs_2
    )

    ,split_month AS (
        SELECT 
            codename, contract_id, booking_id, city, check_in, check_out, 
            explode(
                sequence(
                    trunc(check_in::date, 'month'),
                    trunc(check_out::date, 'month'),
                    INTERVAL 1 month
                )
            ) AS month
            
        FROM rev_wo_vat
    )

    ,ll_per_booking_2 AS (
        SELECT
            s.codename, s.city, s.booking_id, S.MONTH, s.contract_id,
            CASE 
                WHEN s.MONTH = trunc(check_in, 'month') AND s.MONTH = date_trunc('month', check_out) THEN (check_out::DATE - check_in::DATE)::INT
                WHEN s.MONTH = trunc(check_in, 'month') THEN (trunc(check_in, 'month') + interval '1 month' - check_in::DATE)::INT
                WHEN s.MONTH = date_trunc('month', check_out) THEN (check_out::DATE - date_trunc('month', check_out)::DATE)::INT 
                ELSE ((trunc(s.MONTH ,'month') + interval '1 month')::DATE - s.MONTH::DATE)::INT
            END AS days
        FROM split_month s
    )

    ,ll_per_booking AS (
        SELECT 
            s.codename, s.city, s.booking_id,  s.contract_id,
            SUM(CASE WHEN trunc(first_landlord_rent_payment_date,'month') = trunc(s.month, 'month') AND X <= days THEN X
                ELSE days END * ((CASE WHEN landlord::DECIMAL IS NULL THEN 0 ELSE landlord::DECIMAL END)/ CASE WHEN X = 0 THEN 1 ELSE X END)) AS Landlord
        
        FROM ll_per_booking_2 s
            LEFT JOIN landlord_cogs l ON s.MONTH = l.MONTH AND s.codename = l.codename
            
        GROUP BY s.codename, s.city, s.booking_id,  s.contract_id
            
        ORDER BY s.booking_id DESC
    )
    --------------------------------------UNTIL HERE LANDLORD, CONSIDERING GRD AND CARENCIA-----------------

    ,final_2 AS (
        SELECT
            remove_vat.*,
            rev_wo_vat.utilities, 
            rev_wo_vat.final_cleaning,
            rev_wo_vat.pets,
            rev_wo_vat.extra_cleaning,
            rev_wo_vat.in_person_check_in,
            rev_wo_vat.parking,
            rev_wo_vat.deposit,
            rev_wo_vat.other_revenue,
            remove_vat_sc.total_sc,
            ll_per_booking.Landlord AS total_landlord
            
        FROM remove_vat
            LEFT JOIN rev_wo_vat ON rev_wo_vat.contract_id = remove_vat.contract_id
            LEFT JOIN remove_vat_sc ON rev_wo_vat.contract_id = remove_vat_sc.contract_id
            LEFT JOIN ll_per_booking ON rev_wo_vat.contract_id = ll_per_booking.contract_id
    )

    ,final AS (
        SELECT *,
            COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0) AS total_rev,
            total_landlord/nights*(365/12) AS avg_monthly_landlord,
            
            case  ---VAT of total_revenue
                when city = 'Madrid' or city = 'Barcelona' then (COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.1*0.1
                when city = 'Berlin' then (COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.07*0.07
                else 0 
            end as vat_revenue,
            
            case  ---VAT of total_rent
                when city = 'Madrid' or city = 'Barcelona' then total_rent/1.1*0.1
                when city = 'Berlin' then total_rent/1.07*0.07
                else 0 
            end as vat_rent,
            
            case  ---VAT of total_sc
                when city = 'Madrid' or city = 'Barcelona' then total_sc/1.1*0.1
                when city = 'Berlin' then total_sc/1.07*0.07
                else 0 
            end as vat_sc,
            
            case  ---VAT of other revenue concepts
                when city = 'Madrid' or city = 'Barcelona' then (COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.1*0.1
                when city = 'Berlin' then (COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.07*0.07
                else 0 
            end as vat_other_rev
            
        FROM final_2
    )

    ,Est AS (
        SELECT 
            k.contract_id, cl.code, (cl.price::DECIMAL / 100) AS SC_Price 
            
        FROM final k
            LEFT JOIN silver.backoffice.contract_lines cl ON k.contract_id = cl.contract_id 
            
        WHERE code = 'service_charge'
            AND k.city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
    )

    ,X AS (
        SELECT 
            k.codename, k.booking_id, k.city, k.check_in, k.check_out, k.source, k.sell_date, k.contract_id, k.nights, k.total_rent,
            CASE 
                WHEN (city = 'Paris' AND k.sell_date >= DATE('2023-12-04') AND k.total_sc IS NULL) THEN utilities - (CASE WHEN SC_Price IS NULL THEN 0 ELSE SC_Price END)
                ELSE k.utilities
            END AS utilities,
            CASE 
                WHEN (city = 'Paris' AND k.sell_date >= DATE('2023-12-04') AND k.total_sc IS NULL) THEN (CASE WHEN SC_Price IS NULL THEN 0 ELSE SC_Price END)
                ELSE k.total_sc
            END AS total_sc,
            k.final_cleaning, k.pets, k.extra_cleaning, k.in_person_check_in, k.parking, k.deposit, k.other_revenue, k.total_landlord, k.total_rev,
            k.avg_monthly_landlord, k.vat_revenue, k.vat_rent, k.vat_sc,
            k.vat_other_rev
            
        FROM final k LEFT JOIN Est ON k.contract_id = Est.contract_id
    )

    ,k_i AS (
        SELECT *,
            COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent - total_landlord AS total_margin,
            ((COALESCE(total_rent,0) + COALESCE(total_sc,0)) - vat_sc - vat_rent - total_landlord)/(CASE WHEN nights = 0 THEN 1 ELSE nights END)*365/12 AS mg_booked,
            ((COALESCE(total_rent,0) + COALESCE(total_sc,0)) - vat_sc - vat_rent - total_landlord) / (CASE WHEN (COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent) = 0 THEN 1
            ELSE (COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent) END) AS pct_mg
                                                                                                        
        FROM X
            
        WHERE true 
        
        ORDER BY booking_id DESC
    )

    SELECT * FROM k_i
"""

spark.sql(margins_per_extension).write.mode("overwrite").saveAsTable("silver.finance.margins_per_extension")

#### 2.3.1.2. Deltas 7 Days Ago
###### (Margins per Extension)

In [0]:
margins_per_extension_7_days_deltas = """
with 
    payments AS ( 
        SELECT ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
            b.id AS booking_id,
            contracts.inserted_at::DATE AS sell_date, -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.start_date as check_in,
            contracts.id as contract_id,
            contracts.end_date as check_out,
            transactions.id as transaction_id, transaction_items.description,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price::decimal/100::decimal) 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment
        
        from silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b
            left join silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft')
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and (transaction_items.description ILIKE '%rent%')
    )

    ,ttl_rent AS (
    SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        sell_date,
        contract_id,
        SUM(payment) AS total_rent
        FROM payments
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, contract_id
    )

    ,comm AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN total_rent/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN total_rent/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN total_rent/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN total_rent/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN total_rent/1.15
            
            --otas
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN total_rent/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN total_rent/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN total_rent/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_rent/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Berlin', 'Paris') THEN total_rent/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN total_rent/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN total_rent/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN total_rent/1.15
            
            ELSE total_rent
        END AS total_rent, booking_id,city,codename,check_in,check_out,nights,source,sell_date,
        contract_id
        FROM ttl_rent
    )

    ,remove_vat AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        --to avoid divisions by zero I'm adding a case when in the column of nights
        case when nights::integer = 0 then 1 else nights::integer end as nights,
        source,
        sell_date,
        contract_id,
        case 
            when city = 'Madrid' or city = 'Barcelona' then total_rent 
            when city = 'Berlin' then total_rent
            else total_rent 
        end as total_rent
        FROM comm
            WHERE true 
            --AND check_out - check_in >= 365
            --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
        ORDER BY booking_id DESC
    ) 
    ---------------------------------------------UNTIL HERE RENT----------------------------------------------------
    ,payments_sc AS ( 
        SELECT ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
            b.id AS booking_id,
            contracts.inserted_at::DATE AS sell_date, -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.id as contract_id,
            contracts.start_date as check_in,
            contracts.end_date as check_out, transactions.id as transaction_id, transaction_items.description,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price::decimal/100::decimal) 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment
        
        from silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b
            left join silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id

        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft') -------------not including canceled, since no margin gets generated--------
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and (contract_lines.code = 'service_charge'
                or (transaction_items.description ILIKE '%service charge%')
                or (transaction_items.description ILIKE '%service charge%') -----------------check this------------------ops discounts, refunds
                or (transaction_items.description ILIKE '%service_charge%')
                or (transaction_items.description ILIKE '%administration fee%') ----------------check---------------including refunds, penalties
                or (transaction_items.description ILIKE '%admini fee%')
                )
    )

    ,ttl_rent_sc AS (
        SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_in - check_out AS nights,
        source,
        contract_id,
        state,
        sell_date,
        SUM(payment) AS total_sc
        FROM payments_sc
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, state, contract_id
    )

    ,comm_sc AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN total_sc/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN total_sc/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN total_sc/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN total_sc/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN total_sc/1.15
            
            --otas
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN total_sc/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN total_sc/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN total_sc/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_sc/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Berlin', 'Paris') THEN total_sc/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN total_sc/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN total_sc/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN total_sc/1.15
            
            ELSE total_sc
        END AS total_sc,
        booking_id,
        city,
        codename,
        check_in,
        check_out, contract_id,
        check_out - check_in AS nights,
        source,
        state,
        sell_date
        FROM ttl_rent_sc
    )

    ,remove_vat_sc AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source, contract_id,
        state,
        sell_date,
        case 
            when city = 'Madrid' or city = 'Barcelona' then total_sc 
            when city = 'Berlin' then total_sc
            else total_sc 
        end as total_sc
        FROM comm_sc
            WHERE true 
            --AND check_out - check_in >= 365
            --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
        ORDER BY booking_id DESC
    ) 
    ---------------------------------------------UNTIL HERE SC----------------------------------------------------

    ,full_revenue AS ( 
        SELECT 
            b.id AS booking_id,
            b.inserted_at::DATE AS sell_date, 
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.id as contract_id,
            contracts.start_date as check_in,
            contracts.end_date as check_out,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= '10-27-2023' then (transaction_items.price::decimal/100::decimal)/1.02 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment,
            CASE 
                WHEN contract_lines.code = 'utilities' OR contract_lines.code = 'building_utilities' THEN transaction_items.price::decimal/100::decimal
            END AS utilities,
            CASE 
                WHEN contract_lines.code = 'final_cleaning' THEN transaction_items.price::decimal/100::decimal
            END AS final_cleaning,
            CASE 
                WHEN contract_lines.code = 'pets' THEN transaction_items.price::decimal/100::decimal
            END AS pets,
            CASE 
                WHEN contract_lines.code ILIKE '%extra_cleaning%' THEN transaction_items.price::decimal/100::decimal
            END AS extra_cleaning,
            CASE 
                WHEN contract_lines.code = 'airport_transport' THEN transaction_items.price::decimal/100::decimal
            END AS airport_transport,
            CASE 
                WHEN contract_lines.code = 'in_person_check_in' THEN transaction_items.price::decimal/100::decimal
            END AS in_person_check_in,
            CASE 
                WHEN contract_lines.code = 'parking' THEN transaction_items.price::decimal/100::decimal
            END AS parking,
            CASE 
                WHEN contract_lines.code = 'deposit' THEN transaction_items.price::decimal/100::decimal
            END AS deposit,
            
            CASE 
                WHEN contract_lines.code NOT IN ('rent', 'service_charge', 'utilities', 'building_utilities', 'final_cleaning', 'pets', 'airport_transport', 'in_person_check_in', 'parking', 'deposit') AND contract_lines.code NOT ILIKE '%extra_cleaning%' 
                    THEN transaction_items.price::decimal/100::decimal
            END AS other_revenue
        
        from silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b
            left join silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft')
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and contract_lines.code <> 'deposit'
    )

    ,ttl_revenue AS (
        SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        contract_id,
        source,
        state,
        sell_date,
        SUM(utilities) AS utilities,
        SUM(final_cleaning) AS final_cleaning,
        SUM(pets) AS pets,
        SUM(extra_cleaning) AS extra_cleaning,
        SUM(in_person_check_in) AS in_person_check_in,
        SUM(parking) AS parking,
        SUM(deposit) AS deposit,
        SUM(other_revenue) AS other_revenue
        FROM full_revenue
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, contract_id, state
    )

    ,comm_rev AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN utilities/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN utilities/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN utilities/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN utilities/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN utilities/1.15
            
            --otas (all except HA)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN utilities/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN utilities/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN utilities/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN utilities/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN utilities/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN utilities/1.15
            
            ELSE utilities
        END AS utilities,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN final_cleaning/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN final_cleaning/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN final_cleaning/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN final_cleaning/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN final_cleaning/1.15
            
            --otas (only Airbnb)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN final_cleaning/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN final_cleaning/1.067
            
            ELSE final_cleaning
        END AS final_cleaning,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN pets/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN pets/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN pets/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN pets/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN pets/1.15
            
            --otas (only Airbnb)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN pets/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN pets/1.067
            
            ELSE pets
        END AS pets,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN extra_cleaning/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN extra_cleaning/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN extra_cleaning/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN extra_cleaning/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN extra_cleaning/1.15
            
            ELSE extra_cleaning
        END AS extra_cleaning,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN in_person_check_in/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN in_person_check_in/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN in_person_check_in/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN in_person_check_in/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN in_person_check_in/1.15
            
            ELSE in_person_check_in
        END AS in_person_check_in,
        
        other_revenue,
        parking, --not commissionable
        deposit,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        contract_id
        FROM ttl_revenue
    )

    ,rev_wo_vat AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        utilities, 
        contract_id,
        final_cleaning,
        pets,
        extra_cleaning,
        in_person_check_in,
        parking,
        deposit,
        other_revenue
        
        FROM comm_rev
            WHERE true 
        ORDER BY booking_id DESC
    )

    ------------------------------------UNTIL HERE FULL REVENUE-----------------------------------------------------------

    ,landlord_cogs_2 AS (
        SELECT 
            SUM(landlord_daily_rent)::decimal as landlord, 
            trunc(day, 'month') as month, end_of_grace_period::DATE + 1 as first_landlord_rent_payment_date,
            codename, city

        FROM silver.revenue.landlord TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) landlord
            LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON apartments.id = landlord.apartment_id
        
        GROUP BY month, codename, end_of_grace_period, city
    )

    ,landlord_cogs AS (
        SELECT
            codename, city, month, landlord,
            CASE 
                WHEN trunc(first_landlord_rent_payment_date, 'MM') = trunc(month, 'MM') THEN (trunc(month, 'MM') + INTERVAL '1' MONTH - INTERVAL '1' DAY - first_landlord_rent_payment_date::DATE)::INT
                ELSE EXTRACT(DAY FROM trunc(month, 'mm') + INTERVAL '1' MONTH - INTERVAL '1' DAY)::INT
            END AS x, 
            first_landlord_rent_payment_date
        
        FROM landlord_cogs_2
    )

    ,split_month AS (
        SELECT 
            codename, contract_id, booking_id, city, check_in, check_out, 
            explode(
                sequence(
                    trunc(check_in::date, 'month'),
                    trunc(check_out::date, 'month'),
                    INTERVAL 1 month
                )
            ) AS month
            
        FROM rev_wo_vat
    )

    ,ll_per_booking_2 AS (
        SELECT
            s.codename, s.city, s.booking_id, S.MONTH, s.contract_id,
            CASE 
                WHEN s.MONTH = trunc(check_in, 'month') AND s.MONTH = date_trunc('month', check_out) THEN (check_out::DATE - check_in::DATE)::INT
                WHEN s.MONTH = trunc(check_in, 'month') THEN (trunc(check_in, 'month') + interval '1 month' - check_in::DATE)::INT
                WHEN s.MONTH = date_trunc('month', check_out) THEN (check_out::DATE - date_trunc('month', check_out)::DATE)::INT 
                ELSE ((trunc(s.MONTH ,'month') + interval '1 month')::DATE - s.MONTH::DATE)::INT
            END AS days
        FROM split_month s
    )

    ,ll_per_booking AS (
        SELECT 
            s.codename, s.city, s.booking_id,  s.contract_id,
            SUM(CASE WHEN trunc(first_landlord_rent_payment_date,'month') = trunc(s.month, 'month') AND X <= days THEN X
                ELSE days END * ((CASE WHEN landlord::DECIMAL IS NULL THEN 0 ELSE landlord::DECIMAL END)/ CASE WHEN X = 0 THEN 1 ELSE X END)) AS Landlord
        
        FROM ll_per_booking_2 s
            LEFT JOIN landlord_cogs l ON s.MONTH = l.MONTH AND s.codename = l.codename
            
        GROUP BY s.codename, s.city, s.booking_id,  s.contract_id
            
        ORDER BY s.booking_id DESC
    )
    --------------------------------------UNTIL HERE LANDLORD, CONSIDERING GRD AND CARENCIA-----------------

    ,final_2 AS (
        SELECT
            remove_vat.*,
            rev_wo_vat.utilities, 
            rev_wo_vat.final_cleaning,
            rev_wo_vat.pets,
            rev_wo_vat.extra_cleaning,
            rev_wo_vat.in_person_check_in,
            rev_wo_vat.parking,
            rev_wo_vat.deposit,
            rev_wo_vat.other_revenue,
            remove_vat_sc.total_sc,
            ll_per_booking.Landlord AS total_landlord
            
        FROM remove_vat
            LEFT JOIN rev_wo_vat ON rev_wo_vat.contract_id = remove_vat.contract_id
            LEFT JOIN remove_vat_sc ON rev_wo_vat.contract_id = remove_vat_sc.contract_id
            LEFT JOIN ll_per_booking ON rev_wo_vat.contract_id = ll_per_booking.contract_id
    )

    ,final AS (
        SELECT *,
            COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0) AS total_rev,
            total_landlord/nights*(365/12) AS avg_monthly_landlord,
            
            case  ---VAT of total_revenue
                when city = 'Madrid' or city = 'Barcelona' then (COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.1*0.1
                when city = 'Berlin' then (COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.07*0.07
                else 0 
            end as vat_revenue,
            
            case  ---VAT of total_rent
                when city = 'Madrid' or city = 'Barcelona' then total_rent/1.1*0.1
                when city = 'Berlin' then total_rent/1.07*0.07
                else 0 
            end as vat_rent,
            
            case  ---VAT of total_sc
                when city = 'Madrid' or city = 'Barcelona' then total_sc/1.1*0.1
                when city = 'Berlin' then total_sc/1.07*0.07
                else 0 
            end as vat_sc,
            
            case  ---VAT of other revenue concepts
                when city = 'Madrid' or city = 'Barcelona' then (COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.1*0.1
                when city = 'Berlin' then (COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.07*0.07
                else 0 
            end as vat_other_rev
            
        FROM final_2
    )

    ,Est AS (
        SELECT 
            k.contract_id, cl.code, (cl.price::DECIMAL / 100) AS SC_Price 
            
        FROM final k
            LEFT JOIN silver.backoffice.contract_lines TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) cl ON k.contract_id = cl.contract_id 
            
        WHERE code = 'service_charge'
            AND k.city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
    )

    ,X AS (
        SELECT 
            k.codename, k.booking_id, k.city, k.check_in, k.check_out, k.source, k.sell_date, k.contract_id, k.nights, k.total_rent,
            CASE 
                WHEN (city = 'Paris' AND k.sell_date >= DATE('2023-12-04') AND k.total_sc IS NULL) THEN utilities - (CASE WHEN SC_Price IS NULL THEN 0 ELSE SC_Price END)
                ELSE k.utilities
            END AS utilities,
            CASE 
                WHEN (city = 'Paris' AND k.sell_date >= DATE('2023-12-04') AND k.total_sc IS NULL) THEN (CASE WHEN SC_Price IS NULL THEN 0 ELSE SC_Price END)
                ELSE k.total_sc
            END AS total_sc,
            k.final_cleaning, k.pets, k.extra_cleaning, k.in_person_check_in, k.parking, k.deposit, k.other_revenue, k.total_landlord, k.total_rev,
            k.avg_monthly_landlord, k.vat_revenue, k.vat_rent, k.vat_sc,
            k.vat_other_rev
            
        FROM final k LEFT JOIN Est ON k.contract_id = Est.contract_id
    )

    ,k_i AS (
        SELECT *,
            COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent - total_landlord AS total_margin,
            ((COALESCE(total_rent,0) + COALESCE(total_sc,0)) - vat_sc - vat_rent - total_landlord)/(CASE WHEN nights = 0 THEN 1 ELSE nights END)*365/12 AS mg_booked,
            ((COALESCE(total_rent,0) + COALESCE(total_sc,0)) - vat_sc - vat_rent - total_landlord) / (CASE WHEN (COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent) = 0 THEN 1
            ELSE (COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent) END) AS pct_mg
                                                                                                        
        FROM X
            
        WHERE true 
        
        ORDER BY booking_id DESC
    )

    SELECT * FROM k_i
"""

spark.sql(margins_per_extension_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.margins_per_extension_7_days_deltas")

#### 2.3.1.3. Deltas Since Begining of Month
###### (Margins per Extension)

In [0]:
margins_per_extension_begining_month_deltas = """
with 
    payments AS ( 
        SELECT ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
            b.id AS booking_id,
            contracts.inserted_at::DATE AS sell_date, -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.start_date as check_in,
            contracts.id as contract_id,
            contracts.end_date as check_out,
            transactions.id as transaction_id, transaction_items.description,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price::decimal/100::decimal) 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment
        
        from silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b
            left join silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft')
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and (transaction_items.description ILIKE '%rent%')
    )

    ,ttl_rent AS (
    SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        sell_date,
        contract_id,
        SUM(payment) AS total_rent
        FROM payments
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, contract_id
    )

    ,comm AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN total_rent/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN total_rent/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN total_rent/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN total_rent/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN total_rent/1.15
            
            --otas
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN total_rent/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN total_rent/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN total_rent/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_rent/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Berlin', 'Paris') THEN total_rent/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN total_rent/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN total_rent/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN total_rent/1.15
            
            ELSE total_rent
        END AS total_rent, booking_id,city,codename,check_in,check_out,nights,source,sell_date,
        contract_id
        FROM ttl_rent
    )

    ,remove_vat AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        --to avoid divisions by zero I'm adding a case when in the column of nights
        case when nights::integer = 0 then 1 else nights::integer end as nights,
        source,
        sell_date,
        contract_id,
        case 
            when city = 'Madrid' or city = 'Barcelona' then total_rent 
            when city = 'Berlin' then total_rent
            else total_rent 
        end as total_rent
        FROM comm
            WHERE true 
            --AND check_out - check_in >= 365
            --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
        ORDER BY booking_id DESC
    ) 
    ---------------------------------------------UNTIL HERE RENT----------------------------------------------------
    ,payments_sc AS ( 
        SELECT ----------------------where no payment inserted in the BO (no transaction items), margins cannot be calculated------------------
            b.id AS booking_id,
            contracts.inserted_at::DATE AS sell_date, -- OKAY PERO SI HEMOS DE SEPARAR EXTENSIONES DEL BOOKING ORGINAL TENDREMOS QUE UTILIZAR CONTRACTS
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.id as contract_id,
            contracts.start_date as check_in,
            contracts.end_date as check_out, transactions.id as transaction_id, transaction_items.description,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= DATE('2023-10-27') then (transaction_items.price::decimal/100::decimal) 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment
        
        from silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b
            left join silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id

        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft') -------------not including canceled, since no margin gets generated--------
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and (contract_lines.code = 'service_charge'
                or (transaction_items.description ILIKE '%service charge%')
                or (transaction_items.description ILIKE '%service charge%') -----------------check this------------------ops discounts, refunds
                or (transaction_items.description ILIKE '%service_charge%')
                or (transaction_items.description ILIKE '%administration fee%') ----------------check---------------including refunds, penalties
                or (transaction_items.description ILIKE '%admini fee%')
                )
    )

    ,ttl_rent_sc AS (
        SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_in - check_out AS nights,
        source,
        contract_id,
        state,
        sell_date,
        SUM(payment) AS total_sc
        FROM payments_sc
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, state, contract_id
    )

    ,comm_sc AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN total_sc/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN total_sc/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN total_sc/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN total_sc/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN total_sc/1.15
            
            --otas
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN total_sc/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN total_sc/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN total_sc/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Barcelona', 'Madrid', 'Lisbon') THEN total_sc/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Housinganywhere' AND city IN ('Berlin', 'Paris') THEN total_sc/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN total_sc/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN total_sc/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN total_sc/1.15
            
            ELSE total_sc
        END AS total_sc,
        booking_id,
        city,
        codename,
        check_in,
        check_out, contract_id,
        check_out - check_in AS nights,
        source,
        state,
        sell_date
        FROM ttl_rent_sc
    )

    ,remove_vat_sc AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source, contract_id,
        state,
        sell_date,
        case 
            when city = 'Madrid' or city = 'Barcelona' then total_sc 
            when city = 'Berlin' then total_sc
            else total_sc 
        end as total_sc
        FROM comm_sc
            WHERE true 
            --AND check_out - check_in >= 365
            --AND check_in > '01-01-2023' -----------------all times too heavy?-------------------- -- OK HACE SENTIDO
        ORDER BY booking_id DESC
    ) 
    ---------------------------------------------UNTIL HERE SC----------------------------------------------------

    ,full_revenue AS ( 
        SELECT 
            b.id AS booking_id,
            b.inserted_at::DATE AS sell_date, 
            b.source,
            b.state,
            a.codename,
            a.city,
            contracts.id as contract_id,
            contracts.start_date as check_in,
            contracts.end_date as check_out,
            case 
                when contract_lines.code = 'service_charge' and contracts.inserted_at >= '10-27-2023' then (transaction_items.price::decimal/100::decimal)/1.02 
                else (transaction_items.price::decimal/100::decimal)  
            end as payment,
            CASE 
                WHEN contract_lines.code = 'utilities' OR contract_lines.code = 'building_utilities' THEN transaction_items.price::decimal/100::decimal
            END AS utilities,
            CASE 
                WHEN contract_lines.code = 'final_cleaning' THEN transaction_items.price::decimal/100::decimal
            END AS final_cleaning,
            CASE 
                WHEN contract_lines.code = 'pets' THEN transaction_items.price::decimal/100::decimal
            END AS pets,
            CASE 
                WHEN contract_lines.code ILIKE '%extra_cleaning%' THEN transaction_items.price::decimal/100::decimal
            END AS extra_cleaning,
            CASE 
                WHEN contract_lines.code = 'airport_transport' THEN transaction_items.price::decimal/100::decimal
            END AS airport_transport,
            CASE 
                WHEN contract_lines.code = 'in_person_check_in' THEN transaction_items.price::decimal/100::decimal
            END AS in_person_check_in,
            CASE 
                WHEN contract_lines.code = 'parking' THEN transaction_items.price::decimal/100::decimal
            END AS parking,
            CASE 
                WHEN contract_lines.code = 'deposit' THEN transaction_items.price::decimal/100::decimal
            END AS deposit,
            
            CASE 
                WHEN contract_lines.code NOT IN ('rent', 'service_charge', 'utilities', 'building_utilities', 'final_cleaning', 'pets', 'airport_transport', 'in_person_check_in', 'parking', 'deposit') AND contract_lines.code NOT ILIKE '%extra_cleaning%' 
                    THEN transaction_items.price::decimal/100::decimal
            END AS other_revenue
        
        from silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b
            left join silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts on contracts.booking_id = b.id
            left join silver.backoffice.transactions TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transactions on contracts.id = transactions.contract_id
            left join silver.backoffice.transaction_items TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) transaction_items on transactions.id = transaction_items.transaction_id
            left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) a on a.id = b.apartment_id
            left join silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contract_lines on contract_lines.id = transaction_items.contract_line_id
        
        WHERE true 
            AND b.state NOT IN ('denied', 'retracted', 'draft')
            and contracts.status in ( 'booked','partial', 'agreed', 'canceled')
            AND transactions.status not in ('canceled','expired', 'discarded') 
            AND contracts.source ILIKE '%extension%'
            --and transaction_items.price > 0 
            and contract_lines.code <> 'deposit'
    )

    ,ttl_revenue AS (
        SELECT booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        contract_id,
        source,
        state,
        sell_date,
        SUM(utilities) AS utilities,
        SUM(final_cleaning) AS final_cleaning,
        SUM(pets) AS pets,
        SUM(extra_cleaning) AS extra_cleaning,
        SUM(in_person_check_in) AS in_person_check_in,
        SUM(parking) AS parking,
        SUM(deposit) AS deposit,
        SUM(other_revenue) AS other_revenue
        FROM full_revenue
        GROUP BY booking_id, city, codename, check_in, check_out, source, sell_date, contract_id, state
    )

    ,comm_rev AS ( --removing the commission for Apartool bookings inserted after 2/9 or edited manually --check about APPY 13226 and OBOKE 12667
        SELECT 
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN utilities/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN utilities/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN utilities/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN utilities/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN utilities/1.15
            
            --otas (all except HA)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN utilities/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN utilities/1.067
            WHEN sell_date >= '2025-01-01' AND source IN ('Homelike', 'HomeLike') THEN utilities/1.08
            WHEN sell_date >= '2025-01-01' AND source = 'Spotahome' THEN utilities/1.06
            WHEN sell_date >= '2025-01-01' AND source = 'Wunderflats' THEN utilities/1.10
            WHEN sell_date >= '2025-01-01' AND source = 'Plum Guide' THEN utilities/1.15
            
            ELSE utilities
        END AS utilities,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN final_cleaning/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN final_cleaning/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN final_cleaning/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN final_cleaning/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN final_cleaning/1.15
            
            --otas (only Airbnb)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN final_cleaning/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN final_cleaning/1.067
            
            ELSE final_cleaning
        END AS final_cleaning,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN pets/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN pets/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN pets/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN pets/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN pets/1.15
            
            --otas (only Airbnb)
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT <= 90 THEN pets/1.123
            WHEN sell_date >= '2025-01-01' AND source = 'Airbnb' AND nights::INT > 90 THEN pets/1.067
            
            ELSE pets
        END AS pets,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN extra_cleaning/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN extra_cleaning/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN extra_cleaning/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN extra_cleaning/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN extra_cleaning/1.15
            
            ELSE extra_cleaning
        END AS extra_cleaning,
        
        CASE   
            --partners
            WHEN (booking_id IN (13226, 13508, 12787, 12601) OR (sell_date >= DATE('2024-09-02') AND source = 'Apartool')) THEN in_person_check_in/1.15 
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT <= 90 THEN in_person_check_in/1.09
            WHEN sell_date >= '2025-01-01' AND source = 'Altovita' AND nights::INT > 90 THEN in_person_check_in/1.05
            WHEN sell_date >= '2025-01-01' AND source = 'Silverdoor' THEN in_person_check_in/1.15
            WHEN sell_date >= '2025-01-01' AND source = 'StaySitu' THEN in_person_check_in/1.15
            
            ELSE in_person_check_in
        END AS in_person_check_in,
        
        other_revenue,
        parking, --not commissionable
        deposit,
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        check_out - check_in AS nights,
        source,
        state,
        sell_date,
        contract_id
        FROM ttl_revenue
    )

    ,rev_wo_vat AS (
        SELECT  
        booking_id,
        city,
        codename,
        check_in,
        check_out,
        nights,
        source,
        state,
        sell_date,
        utilities, 
        contract_id,
        final_cleaning,
        pets,
        extra_cleaning,
        in_person_check_in,
        parking,
        deposit,
        other_revenue
        
        FROM comm_rev
            WHERE true 
        ORDER BY booking_id DESC
    )

    ------------------------------------UNTIL HERE FULL REVENUE-----------------------------------------------------------

    ,landlord_cogs_2 AS (
        SELECT 
            SUM(landlord_daily_rent)::decimal as landlord, 
            trunc(day, 'month') as month, end_of_grace_period::DATE + 1 as first_landlord_rent_payment_date,
            codename, city

        FROM silver.revenue.landlord TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) landlord
            LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON apartments.id = landlord.apartment_id
        
        GROUP BY month, codename, end_of_grace_period, city
    )

    ,landlord_cogs AS (
        SELECT
            codename, city, month, landlord,
            CASE 
                WHEN trunc(first_landlord_rent_payment_date, 'MM') = trunc(month, 'MM') THEN (trunc(month, 'MM') + INTERVAL '1' MONTH - INTERVAL '1' DAY - first_landlord_rent_payment_date::DATE)::INT
                ELSE EXTRACT(DAY FROM trunc(month, 'mm') + INTERVAL '1' MONTH - INTERVAL '1' DAY)::INT
            END AS x, 
            first_landlord_rent_payment_date
        
        FROM landlord_cogs_2
    )

    ,split_month AS (
        SELECT 
            codename, contract_id, booking_id, city, check_in, check_out, 
            explode(
                sequence(
                    trunc(check_in::date, 'month'),
                    trunc(check_out::date, 'month'),
                    INTERVAL 1 month
                )
            ) AS month
            
        FROM rev_wo_vat
    )

    ,ll_per_booking_2 AS (
        SELECT
            s.codename, s.city, s.booking_id, S.MONTH, s.contract_id,
            CASE 
                WHEN s.MONTH = trunc(check_in, 'month') AND s.MONTH = date_trunc('month', check_out) THEN (check_out::DATE - check_in::DATE)::INT
                WHEN s.MONTH = trunc(check_in, 'month') THEN (trunc(check_in, 'month') + interval '1 month' - check_in::DATE)::INT
                WHEN s.MONTH = date_trunc('month', check_out) THEN (check_out::DATE - date_trunc('month', check_out)::DATE)::INT 
                ELSE ((trunc(s.MONTH ,'month') + interval '1 month')::DATE - s.MONTH::DATE)::INT
            END AS days
        FROM split_month s
    )

    ,ll_per_booking AS (
        SELECT 
            s.codename, s.city, s.booking_id,  s.contract_id,
            SUM(CASE WHEN trunc(first_landlord_rent_payment_date,'month') = trunc(s.month, 'month') AND X <= days THEN X
                ELSE days END * ((CASE WHEN landlord::DECIMAL IS NULL THEN 0 ELSE landlord::DECIMAL END)/ CASE WHEN X = 0 THEN 1 ELSE X END)) AS Landlord
        
        FROM ll_per_booking_2 s
            LEFT JOIN landlord_cogs l ON s.MONTH = l.MONTH AND s.codename = l.codename
            
        GROUP BY s.codename, s.city, s.booking_id,  s.contract_id
            
        ORDER BY s.booking_id DESC
    )
    --------------------------------------UNTIL HERE LANDLORD, CONSIDERING GRD AND CARENCIA-----------------

    ,final_2 AS (
        SELECT
            remove_vat.*,
            rev_wo_vat.utilities, 
            rev_wo_vat.final_cleaning,
            rev_wo_vat.pets,
            rev_wo_vat.extra_cleaning,
            rev_wo_vat.in_person_check_in,
            rev_wo_vat.parking,
            rev_wo_vat.deposit,
            rev_wo_vat.other_revenue,
            remove_vat_sc.total_sc,
            ll_per_booking.Landlord AS total_landlord
            
        FROM remove_vat
            LEFT JOIN rev_wo_vat ON rev_wo_vat.contract_id = remove_vat.contract_id
            LEFT JOIN remove_vat_sc ON rev_wo_vat.contract_id = remove_vat_sc.contract_id
            LEFT JOIN ll_per_booking ON rev_wo_vat.contract_id = ll_per_booking.contract_id
    )

    ,final AS (
        SELECT *,
            COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0) AS total_rev,
            total_landlord/nights*(365/12) AS avg_monthly_landlord,
            
            case  ---VAT of total_revenue
                when city = 'Madrid' or city = 'Barcelona' then (COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.1*0.1
                when city = 'Berlin' then (COALESCE(total_rent,0)+COALESCE(total_sc,0)+COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.07*0.07
                else 0 
            end as vat_revenue,
            
            case  ---VAT of total_rent
                when city = 'Madrid' or city = 'Barcelona' then total_rent/1.1*0.1
                when city = 'Berlin' then total_rent/1.07*0.07
                else 0 
            end as vat_rent,
            
            case  ---VAT of total_sc
                when city = 'Madrid' or city = 'Barcelona' then total_sc/1.1*0.1
                when city = 'Berlin' then total_sc/1.07*0.07
                else 0 
            end as vat_sc,
            
            case  ---VAT of other revenue concepts
                when city = 'Madrid' or city = 'Barcelona' then (COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.1*0.1
                when city = 'Berlin' then (COALESCE(utilities,0)+COALESCE(final_cleaning,0)+COALESCE(pets,0)+COALESCE(extra_cleaning,0)+COALESCE(in_person_check_in,0)+COALESCE(parking,0)+COALESCE(other_revenue,0))/1.07*0.07
                else 0 
            end as vat_other_rev
            
        FROM final_2
    )

    ,Est AS (
        SELECT 
            k.contract_id, cl.code, (cl.price::DECIMAL / 100) AS SC_Price 
            
        FROM final k
            LEFT JOIN silver.backoffice.contract_lines TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) cl ON k.contract_id = cl.contract_id 
            
        WHERE code = 'service_charge'
            AND k.city = 'Paris'
            AND k.sell_date >= DATE('2023-12-04')
    )

    ,X AS (
        SELECT 
            k.codename, k.booking_id, k.city, k.check_in, k.check_out, k.source, k.sell_date, k.contract_id, k.nights, k.total_rent,
            CASE 
                WHEN (city = 'Paris' AND k.sell_date >= DATE('2023-12-04') AND k.total_sc IS NULL) THEN utilities - (CASE WHEN SC_Price IS NULL THEN 0 ELSE SC_Price END)
                ELSE k.utilities
            END AS utilities,
            CASE 
                WHEN (city = 'Paris' AND k.sell_date >= DATE('2023-12-04') AND k.total_sc IS NULL) THEN (CASE WHEN SC_Price IS NULL THEN 0 ELSE SC_Price END)
                ELSE k.total_sc
            END AS total_sc,
            k.final_cleaning, k.pets, k.extra_cleaning, k.in_person_check_in, k.parking, k.deposit, k.other_revenue, k.total_landlord, k.total_rev,
            k.avg_monthly_landlord, k.vat_revenue, k.vat_rent, k.vat_sc,
            k.vat_other_rev
            
        FROM final k LEFT JOIN Est ON k.contract_id = Est.contract_id
    )

    ,k_i AS (
        SELECT *,
            COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent - total_landlord AS total_margin,
            ((COALESCE(total_rent,0) + COALESCE(total_sc,0)) - vat_sc - vat_rent - total_landlord)/(CASE WHEN nights = 0 THEN 1 ELSE nights END)*365/12 AS mg_booked,
            ((COALESCE(total_rent,0) + COALESCE(total_sc,0)) - vat_sc - vat_rent - total_landlord) / (CASE WHEN (COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent) = 0 THEN 1
            ELSE (COALESCE(total_rent,0) + COALESCE(total_sc,0) - vat_sc - vat_rent) END) AS pct_mg
                                                                                                        
        FROM X
            
        WHERE true 
        
        ORDER BY booking_id DESC
    )

    SELECT * FROM k_i
"""

spark.sql(margins_per_extension_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.margins_per_extension_begining_month_deltas")

### 2.3.2. Extension Masters

#### 2.3.2.1. Actuals
###### (Extensions Master)

In [0]:
extensions_masters = """
with 
    refer AS (
        SELECT 
            apartments.city, 
            bookings.source as source_drilldown_1, 
            cc.source as source_drilldown, 
            apartments.codename, 
            apartments.category,
            apartments.neighbourhood, 
            c.booking_id as booking_id, 
            cc.id as extension_id, 
            min(c.start_date) as original_contract_start_date, 
            min(c.end_date) as original_check_out,
            CASE WHEN cc.start_date > bookings.check_out THEN bookings.check_out ELSE cc.start_date END AS extension_start_date, 
            CASE WHEN cc.end_date > bookings.check_out THEN bookings.check_out ELSE cc.end_date END AS extension_end_date,
            cc.inserted_at as extension_created_at,
            CASE 
                WHEN cc.end_date > bookings.check_out THEN 'ED Extension'
                ELSE 'Extension'
            END AS ext_type
        
        FROM silver.backoffice.contracts c
            LEFT JOIN silver.backoffice.contracts cc ON c.booking_id = cc.booking_id 
            LEFT JOIN silver.backoffice.bookings ON c.booking_id = bookings.id
            LEFT JOIN silver.backoffice.apartments ON bookings.apartment_id= apartments.id
        
        WHERE true
            AND c.status IN ('booked', 'partial') 
            AND cc.status in('booked','partial') --??
            AND cc.source ILIKE '%extension%'

        
        GROUP BY
            apartments.city, apartments.codename,
            apartments.category, apartments.neighbourhood,
            c.booking_id, bookings.source, cc.source,
            cc.id, cc.start_date,
            cc.end_date, cc.inserted_at,
            bookings.check_out
            
        ORDER BY cc.inserted_at DESC
    ) 

    ,extensions as(
        SELECT 
            extension_created_at, 
            extension_id,
            source_drilldown_1, 
            count(*), 
            source_drilldown,
            booking_id, 
            extension_start_date, 
            extension_end_date, 
            original_check_out,
            ext_type
        
        from refer

        group by 
            date_trunc('month', extension_created_at), extension_id,
            extension_created_at, booking_id, source_drilldown,
            extension_end_date, extension_start_date, 
            original_check_out, source_drilldown_1,
            ext_type
    )

    ,z as (
        select * from extensions 
    )


    ----------------------------------------------------------------
    -------UNION WITH BOOKINGS MASTERS + ADD THE SAME FORMAT -------
    ----------------------------------------------------------------

    ,bookings_master as (
        SELECT * FROM silver.finance.bookings_masters
    )

    ,margins as (
        SELECT * FROM silver.finance.margins_per_extension
    )

    ,w as (
        SELECT  *
        FROM z 
            LEFT JOIN bookings_master ON bookings_master.id = z.booking_id
    )

    ,x as (
        SELECT 
            w.id, w.extension_start_date as check_in, w.extension_end_date as check_out, w.guest_id, w.extension_created_at as sell_date,
            w.source_drilldown_1 as bookings_source, w.source_drilldown as contracts_source, (w.extension_end_date-w.extension_start_date) as nights, w.codename, w.city, 
            w.name, w.last_name, w.email, w.stay_reason, w.state, margins.total_rev, margins.total_rent, margins.total_landlord, margins.total_margin, margins.mg_booked, margins.total_sc, w.extension_id as contract_id, margins.utilities
            
        FROM w 
            LEFT JOIN margins ON margins.booking_id = w.id AND margins.contract_id = w.extension_id
        
        ORDER BY w.extension_created_at DESC
    )

    ,test as (
        SELECT x.*
        
        FROM x 
        
        WHERE true
        
        ORDER BY x.contract_id DESC
    )

    ,A AS (
        SELECT * FROM silver.finance.overlapping
    )

    ,B AS (
        SELECT 
            test.*, MIN(a.ci) ci, MIN(a.co) co, test.sell_date as inserted_at_2 
        
        FROM test 
            LEFT JOIN A ON a.CONTRACT_ID_FILLED = test.contract_id
        
        GROUP BY 
            test.id, test.check_in, 
            test.sell_date, test.check_out, 
            test.codename, test.city, stay_reason, 
            state, name, last_name, email, bookings_source, 
            contracts_source, guest_id, contract_id, nights, total_rev, 
            total_rent, total_sc, utilities,
            total_landlord, mg_booked, total_margin, a.inserted_at
    )

    ,C AS (
        SELECT 
            id, COALESCE(ci, check_in) check_in, 
            CASE WHEN inserted_at_2 > check_in THEN check_in ELSE inserted_at_2 END AS sell_date, 
            COALESCE(co,check_out) check_out, codename, city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities,
            total_landlord, mg_booked, total_margin
        
        FROM B
    )


    SELECT id, check_in, check_out, sell_date, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, total_landlord, mg_booked, total_margin FROM C WHERE sell_date >= DATE('2025-02-01')

    UNION ALL 

    SELECT id, check_in, check_out, sell_date, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, total_landlord, mg_booked, total_margin FROM silver.finance.materialized_extensions_masters
"""

spark.sql(extensions_masters).write.mode("overwrite").saveAsTable("silver.finance.extensions_masters")

#### 2.3.2.2. Deltas 7 Days Ago
###### (Extensions Master)

In [0]:
extensions_masters_7_days_deltas = """
with 
    refer AS (
        SELECT 
            apartments.city, 
            bookings.source as source_drilldown_1, 
            cc.source as source_drilldown, 
            apartments.codename, 
            apartments.category,
            apartments.neighbourhood, 
            c.booking_id as booking_id, 
            cc.id as extension_id, 
            min(c.start_date) as original_contract_start_date, 
            min(c.end_date) as original_check_out,
            CASE WHEN cc.start_date > bookings.check_out THEN bookings.check_out ELSE cc.start_date END AS extension_start_date, 
            CASE WHEN cc.end_date > bookings.check_out THEN bookings.check_out ELSE cc.end_date END AS extension_end_date,
            cc.inserted_at as extension_created_at,
            CASE 
                WHEN cc.end_date > bookings.check_out THEN 'ED Extension'
                ELSE 'Extension'
            END AS ext_type
        
        FROM silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) c
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) cc ON c.booking_id = cc.booking_id 
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON c.booking_id = bookings.id
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON bookings.apartment_id = apartments.id
        
        WHERE true
            AND c.status IN ('booked', 'partial') 
            AND cc.status in('booked','partial') --??
            AND cc.source ILIKE '%extension%'

        
        GROUP BY
            apartments.city, apartments.codename,
            apartments.category, apartments.neighbourhood,
            c.booking_id, bookings.source, cc.source,
            cc.id, cc.start_date,
            cc.end_date, cc.inserted_at,
            bookings.check_out
            
        ORDER BY cc.inserted_at DESC
    ) 

    ,extensions as(
        SELECT 
            extension_created_at, 
            extension_id,
            source_drilldown_1, 
            count(*), 
            source_drilldown,
            booking_id, 
            extension_start_date, 
            extension_end_date, 
            original_check_out,
            ext_type
        
        from refer

        group by 
            date_trunc('month', extension_created_at), extension_id,
            extension_created_at, booking_id, source_drilldown,
            extension_end_date, extension_start_date, 
            original_check_out, source_drilldown_1,
            ext_type
    )

    ,z as (
        select * from extensions 
    )


    ----------------------------------------------------------------
    -------UNION WITH BOOKINGS MASTERS + ADD THE SAME FORMAT -------
    ----------------------------------------------------------------

    ,bookings_master as (
        SELECT * FROM silver.finance.bookings_masters_7_days_deltas
    )

    ,margins as (
        SELECT * FROM silver.finance.margins_per_extension_7_days_deltas
    )

    ,w as (
        SELECT  *
        FROM z 
            LEFT JOIN bookings_master ON bookings_master.id = z.booking_id
    )

    ,x as (
        SELECT 
            w.id, w.extension_start_date as check_in, w.extension_end_date as check_out, w.guest_id, w.extension_created_at as sell_date,
            w.source_drilldown_1 as bookings_source, w.source_drilldown as contracts_source, (w.extension_end_date-w.extension_start_date) as nights, w.codename, w.city, 
            w.name, w.last_name, w.email, w.stay_reason, w.state, margins.total_rev, margins.total_rent, margins.total_landlord, margins.total_margin, margins.mg_booked, margins.total_sc, w.extension_id as contract_id, margins.utilities
            
        FROM w 
            LEFT JOIN margins ON margins.booking_id = w.id AND margins.contract_id = w.extension_id
        
        ORDER BY w.extension_created_at DESC
    )

    ,test as (
        SELECT x.*
        
        FROM x 
        
        WHERE true
        
        ORDER BY x.contract_id DESC
    )

    ,A AS (
        SELECT * FROM silver.finance.overlapping
    )

    ,B AS (
        SELECT 
            test.*, MIN(a.ci) ci, MIN(a.co) co, test.sell_date as inserted_at_2 
        
        FROM test 
            LEFT JOIN A ON a.CONTRACT_ID_FILLED = test.contract_id
        
        GROUP BY 
            test.id, test.check_in, 
            test.sell_date, test.check_out, 
            test.codename, test.city, stay_reason, 
            state, name, last_name, email, bookings_source, 
            contracts_source, guest_id, contract_id, nights, total_rev, 
            total_rent, total_sc, utilities,
            total_landlord, mg_booked, total_margin, a.inserted_at
    )

    ,C AS (
        SELECT 
            id, COALESCE(ci, check_in) check_in, 
            CASE WHEN inserted_at_2 > check_in THEN check_in ELSE inserted_at_2 END AS sell_date, 
            COALESCE(co,check_out) check_out, codename, city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities,
            total_landlord, mg_booked, total_margin
        
        FROM B
    )


    SELECT id, check_in, check_out, sell_date, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, total_landlord, mg_booked, total_margin FROM C
"""

spark.sql(extensions_masters_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.extensions_masters_7_days_deltas")

#### 2.3.2.3. Deltas Since Begining of Month
###### (Margins per Extension)

In [0]:
extensions_masters_begining_month_deltas = """
with 
    refer AS (
        SELECT 
            apartments.city, 
            bookings.source as source_drilldown_1, 
            cc.source as source_drilldown, 
            apartments.codename, 
            apartments.category,
            apartments.neighbourhood, 
            c.booking_id as booking_id, 
            cc.id as extension_id, 
            min(c.start_date) as original_contract_start_date, 
            min(c.end_date) as original_check_out,
            CASE WHEN cc.start_date > bookings.check_out THEN bookings.check_out ELSE cc.start_date END AS extension_start_date, 
            CASE WHEN cc.end_date > bookings.check_out THEN bookings.check_out ELSE cc.end_date END AS extension_end_date,
            cc.inserted_at as extension_created_at,
            CASE 
                WHEN cc.end_date > bookings.check_out THEN 'ED Extension'
                ELSE 'Extension'
            END AS ext_type
        
        FROM silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) c
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) cc ON c.booking_id = cc.booking_id 
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON c.booking_id = bookings.id
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON bookings.apartment_id = apartments.id
        
        WHERE true
            AND c.status IN ('booked', 'partial') 
            AND cc.status in('booked','partial') --??
            AND cc.source ILIKE '%extension%'

        
        GROUP BY
            apartments.city, apartments.codename,
            apartments.category, apartments.neighbourhood,
            c.booking_id, bookings.source, cc.source,
            cc.id, cc.start_date,
            cc.end_date, cc.inserted_at,
            bookings.check_out
            
        ORDER BY cc.inserted_at DESC
    ) 

    ,extensions as(
        SELECT 
            extension_created_at, 
            extension_id,
            source_drilldown_1, 
            count(*), 
            source_drilldown,
            booking_id, 
            extension_start_date, 
            extension_end_date, 
            original_check_out,
            ext_type
        
        from refer

        group by 
            date_trunc('month', extension_created_at), extension_id,
            extension_created_at, booking_id, source_drilldown,
            extension_end_date, extension_start_date, 
            original_check_out, source_drilldown_1,
            ext_type
    )

    ,z as (
        select * from extensions 
    )


    ----------------------------------------------------------------
    -------UNION WITH BOOKINGS MASTERS + ADD THE SAME FORMAT -------
    ----------------------------------------------------------------

    ,bookings_master as (
        SELECT * FROM silver.finance.bookings_masters_begining_month_deltas
    )

    ,margins as (
        SELECT * FROM silver.finance.margins_per_extension_begining_month_deltas
    )

    ,w as (
        SELECT  *
        FROM z 
            LEFT JOIN bookings_master ON bookings_master.id = z.booking_id
    )

    ,x as (
        SELECT 
            w.id, w.extension_start_date as check_in, w.extension_end_date as check_out, w.guest_id, w.extension_created_at as sell_date,
            w.source_drilldown_1 as bookings_source, w.source_drilldown as contracts_source, (w.extension_end_date-w.extension_start_date) as nights, w.codename, w.city, 
            w.name, w.last_name, w.email, w.stay_reason, w.state, margins.total_rev, margins.total_rent, margins.total_landlord, margins.total_margin, margins.mg_booked, margins.total_sc, w.extension_id as contract_id, margins.utilities
            
        FROM w 
            LEFT JOIN margins ON margins.booking_id = w.id AND margins.contract_id = w.extension_id
        
        ORDER BY w.extension_created_at DESC
    )

    ,test as (
        SELECT x.*
        
        FROM x 
        
        WHERE true
        
        ORDER BY x.contract_id DESC
    )

    ,A AS (
        SELECT * FROM silver.finance.overlapping
    )

    ,B AS (
        SELECT 
            test.*, MIN(a.ci) ci, MIN(a.co) co, test.sell_date as inserted_at_2 
        
        FROM test 
            LEFT JOIN A ON a.CONTRACT_ID_FILLED = test.contract_id
        
        GROUP BY 
            test.id, test.check_in, 
            test.sell_date, test.check_out, 
            test.codename, test.city, stay_reason, 
            state, name, last_name, email, bookings_source, 
            contracts_source, guest_id, contract_id, nights, total_rev, 
            total_rent, total_sc, utilities,
            total_landlord, mg_booked, total_margin, a.inserted_at
    )

    ,C AS (
        SELECT 
            id, COALESCE(ci, check_in) check_in, 
            CASE WHEN inserted_at_2 > check_in THEN check_in ELSE inserted_at_2 END AS sell_date, 
            COALESCE(co,check_out) check_out, codename, city, stay_reason, state, name, last_name, email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, total_sc, utilities,
            total_landlord, mg_booked, total_margin
        
        FROM B
    )


    SELECT id, check_in, check_out, sell_date, codename, city, stay_reason, state, name, last_name, 
    email, bookings_source, contracts_source, guest_id, contract_id, nights, total_rev, total_rent, 
    total_sc, utilities, total_landlord, mg_booked, total_margin FROM C
"""

spark.sql(extensions_masters_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.extensions_masters_begining_month_deltas")

## 2.4. New Sales & Sources (Booking Details)

### 2.4.1. New Sales & Sources

#### 2.4.1.1. Actuals
###### (New Sales & Sources)

In [0]:
new_sales_and_source = """
WITH 
    BM AS (
        SELECT * FROM silver.finance.bookings_masters
    )

    ,BOE AS (
        SELECT * FROM silver.finance.extensions_masters
    )

    ,X AS (
        SELECT 
            total_rent, BM.id, check_in, check_out, (check_out - check_in) nights, apartments.codename, BM.city, 
            bookings_source, contracts_source, total_sc, utilities, final_cleaning, 'Original Booking' type, bm.inserted_at sell_date, 
            (extra_cleaning + pets + in_person_check_in + other_revenue) AS other_revenue, deposit, contract_id
            
        FROM BM
        left join silver.backoffice.apartments on apartments.codename = BM.codename
        where true 
        
        UNION ALL
        
        SELECT 
            total_rent, BOE.id, check_in, check_out, (check_out - check_in) nights, apartments.codename, BOE.city, 
            bookings_source, contracts_source, total_sc, utilities, 0 AS final_cleaning, 'Extension' type, sell_date,
            0 AS other_revenue, 0 AS deposit, contract_id
        
        FROM BOE
        left join silver.backoffice.apartments on apartments.codename = BOE.codename 
        where true 
    )

    SELECT 
        total_rent, total_sc, utilities, final_cleaning, id, codename, city, sell_date,
        check_in, check_out, type, bookings_source, contracts_source, other_revenue other_revenue, nights::int, deposit,
        (COALESCE(total_rent,0) + COALESCE(total_sc,0) + COALESCE(utilities,0) + COALESCE(final_cleaning,0) + COALESCE(other_revenue,0)) total_rev, contract_id 
    FROM X 

    ORDER BY sell_date DESC
"""

spark.sql(new_sales_and_source).write.mode("overwrite").saveAsTable("silver.finance.new_sales_and_source")

#### 2.4.1.2. Deltas 7 Days Ago
###### (New Sales & Sources)

In [0]:
new_sales_and_source_7_days_deltas = """
WITH 
    BM AS (
        SELECT * FROM silver.finance.bookings_masters_7_days_deltas
    )

    ,BOE AS (
        SELECT * FROM silver.finance.extensions_masters_7_days_deltas
    )

    ,X AS (
        SELECT 
            total_rent, BM.id, check_in, check_out, (check_out - check_in) nights, apartments.codename, BM.city, 
            bookings_source, contracts_source, total_sc, utilities, final_cleaning, 'Original Booking' type, bm.inserted_at sell_date, 
            (extra_cleaning + pets + in_person_check_in + other_revenue) AS other_revenue, deposit, contract_id
            
        FROM BM
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments on apartments.codename = BM.codename
        where true 
        
        UNION ALL
        
        SELECT 
            total_rent, BOE.id, check_in, check_out, (check_out - check_in) nights, apartments.codename, BOE.city, 
            bookings_source, contracts_source, total_sc, utilities, 0 AS final_cleaning, 'Extension' type, sell_date,
            0 AS other_revenue, 0 AS deposit, contract_id
        
        FROM BOE
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments on apartments.codename = BOE.codename 
        where true 
    )

    SELECT 
        total_rent, total_sc, utilities, final_cleaning, id, codename, city, sell_date,
        check_in, check_out, type, bookings_source, contracts_source, other_revenue other_revenue, nights::int, deposit,
        (COALESCE(total_rent,0) + COALESCE(total_sc,0) + COALESCE(utilities,0) + COALESCE(final_cleaning,0) + COALESCE(other_revenue,0)) total_rev, contract_id 
    FROM X 

    ORDER BY sell_date DESC
"""

spark.sql(new_sales_and_source_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.new_sales_and_source_7_days_deltas")

#### 2.4.1.3. Deltas Since Begining of Month
###### (New Sales & Sources)

In [0]:
new_sales_and_source_begining_month_deltas = """
WITH 
    BM AS (
        SELECT * FROM silver.finance.bookings_masters_begining_month_deltas
    )

    ,BOE AS (
        SELECT * FROM silver.finance.extensions_masters_begining_month_deltas
    )

    ,X AS (
        SELECT 
            total_rent, BM.id, check_in, check_out, (check_out - check_in) nights, apartments.codename, BM.city, 
            bookings_source, contracts_source, total_sc, utilities, final_cleaning, 'Original Booking' type, bm.inserted_at sell_date, 
            (extra_cleaning + pets + in_person_check_in + other_revenue) AS other_revenue, deposit, contract_id
            
        FROM BM
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments on apartments.codename = BM.codename
        where true 
        
        UNION ALL
        
        SELECT 
            total_rent, BOE.id, check_in, check_out, (check_out - check_in) nights, apartments.codename, BOE.city, 
            bookings_source, contracts_source, total_sc, utilities, 0 AS final_cleaning, 'Extension' type, sell_date,
            0 AS other_revenue, 0 AS deposit, contract_id
        
        FROM BOE
        left join silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments on apartments.codename = BOE.codename 
        where true 
    )

    SELECT 
        total_rent, total_sc, utilities, final_cleaning, id, codename, city, sell_date,
        check_in, check_out, type, bookings_source, contracts_source, other_revenue other_revenue, nights::int, deposit,
        (COALESCE(total_rent,0) + COALESCE(total_sc,0) + COALESCE(utilities,0) + COALESCE(final_cleaning,0) + COALESCE(other_revenue,0)) total_rev, contract_id 
    FROM X 

    ORDER BY sell_date DESC
"""

spark.sql(new_sales_and_source_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.new_sales_and_source_begining_month_deltas")

### 2.4.2. Ocupied Nights per Booking


#### 2.4.2.1. Actuals
###### (Occupied Nights per Booking)

In [0]:
Occupied_Nights_per_Booking = """
WITH Basic AS (
        SELECT id, check_in, check_out, sell_date, bookings_source, contracts_source,
        nights, codename, city, type, contract_id FROM silver.finance.new_sales_and_source 
    )

    ,explode_months AS (
        SELECT
            id, sell_date, type, bookings_source, contracts_source, codename, contract_id, city, check_in, check_out, nights,
            explode(
                sequence(
                    trunc(check_in, 'month'),
                    trunc(check_out, 'month'),
                    INTERVAL 1 MONTH    
                )
            ) AS month_date

        FROM Basic ORDER BY id DESC, month_Date DESC
    )

    ,prefinal AS (
        SELECT
            id, month_date, sell_date, codename, city, bookings_source, contracts_source, type, contract_id, check_in, check_out,
            CASE
                WHEN trunc(check_in, 'month') = month_date AND trunc(check_out, 'month') = month_date 
                    THEN (check_out - check_in)::INT
                WHEN trunc(check_in, 'month') = month_date 
                    THEN EXTRACT(DAY FROM trunc(month_date, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY) - EXTRACT(DAY FROM check_in) + 1
                WHEN trunc(check_out, 'month') = month_date 
                    THEN EXTRACT(DAY FROM check_out) - 1
                ELSE EXTRACT(DAY FROM trunc(month_date, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY)
            END AS occupied_nights

        FROM explode_months
    )

    SELECT * FROM prefinal
"""

spark.sql(Occupied_Nights_per_Booking).write.mode("overwrite").saveAsTable("silver.finance.Occupied_Nights_per_Booking")

# 

#3. OCCUPANCY

## 3.1. Bookable & Booked

### 3.1.1. Actuals
###### (Bookable & Booked)

In [0]:
bookable_and_booked = """
WITH 

    Base_Bookable AS (
        SELECT count(day) as bookable_1, trunc(day, 'month') month, apartment_id, guest_ready_date FROM silver.revenue.occupancy GROUP BY trunc(day, 'month'), apartment_id, guest_ready_date
    )

    ,Blocked_not_occ AS (
        SELECT count(IS_BLOCKED) non_bookable, trunc(day, 'month') month, apartment_id FROM silver.revenue.occupancy WHERE IS_BLOCKED 
                AND  (array_contains(BLOCK_REASON, 'RFBS')
                        OR  array_contains(BLOCK_REASON, 'MNTL')
                        OR  array_contains(BLOCK_REASON, 'QCHK')
                        OR  array_contains(BLOCK_REASON, 'PETS')
                        OR  array_contains(BLOCK_REASON, 'CLNG')
                        OR  array_contains(BLOCK_REASON, 'HOLD')
                        OR  array_contains(BLOCK_REASON, 'MNTE')
                        OR  array_contains(BLOCK_REASON, 'GRDT')
                        OR  array_contains(BLOCK_REASON, 'BTOB')
                        OR  array_contains(BLOCK_REASON, 'RLCT')
                        OR  array_contains(BLOCK_REASON, 'XOTH')
                        OR  array_contains(BLOCK_REASON, 'DSGN')
                )
                --AND CAST(BLOCK_REASON as string) <> '[SALS]'
        GROUP BY trunc(day, 'month'), apartment_id
    )

    ,Bookable_2 AS (
        SELECT 
            bb.month, bookable_1 as bookable, bookable_1 as sales_bookable, bb.apartment_id, guest_ready_date,
            JSON_TUPLE(CONTRACT_INFO, 'end_of_contract') e_o_contract
        FROM Base_Bookable bb 
            LEFT JOIN silver.backoffice.ams_contract_terms ON backoffice.ams_contract_terms.apartment_id = bb.apartment_id
        ORDER BY bb.month DESC
    )

    ,Bookable_4 AS (
        SELECT 
            month, apartment_id,
            CASE 
                WHEN month > trunc(e_o_contract, 'month') THEN 0
                WHEN month = trunc(e_o_contract, 'month') THEN EXTRACT(DAY FROM e_o_contract)::INT
                ELSE bookable
            END AS bookable,
            CASE 
                WHEN month < trunc(guest_ready_date, 'month') THEN 0
                WHEN month = trunc(guest_ready_date, 'month') THEN (EXTRACT(DAY FROM (trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY))::INT) - (EXTRACT(DAY FROM  guest_ready_date)::INT)
                WHEN month > trunc(e_o_contract, 'month') THEN 0
                WHEN month = trunc(e_o_contract, 'month') THEN EXTRACT(DAY FROM e_o_contract)::INT
                ELSE sales_bookable
            END AS sales_bookable, guest_ready_date

        FROM bookable_2
    )

    ,Bookable_3 AS (
        SELECT 
            bookable_4.month, bookable_4.apartment_id, bookable_4.bookable, 
            CASE WHEN (bookable_4.sales_bookable - COALESCE(non_bookable,0)) < 0 THEN 0
                ELSE (bookable_4.sales_bookable - COALESCE(non_bookable,0)) END AS sales_bookable

        FROM bookable_4 bookable_4
            LEFT JOIN Blocked_not_occ bno ON bookable_4.month = bno.month AND bookable_4.apartment_id = bno.apartment_id
    )

    ,Bookable AS (
        SELECT SUM(Bookable) Bookable, SUM(sales_bookable) sales_bookable, month, apartment_id FROM Bookable_3 GROUP BY month, apartment_id
    )
    ,Booked AS (
        SELECT count(IS_BOOKED) booked, trunc(day, 'month') month, apartment_id FROM silver.revenue.occupancy WHERE IS_BOOKED = true GROUP BY trunc(day, 'month'), apartment_id
    )

    ,New AS (
        SELECT 
            bookable.*, COALESCE(booked.booked,0) booked,
            CAST(CAST(COALESCE(booked.booked,0) AS DECIMAL(20, 4)) / NULLIF(CAST(bookable.bookable AS DECIMAL(20, 4)), 0) AS DECIMAL(20, 4)) AS occupancy,
            CAST(CAST(COALESCE(booked.booked,0) AS DECIMAL(20, 4)) / NULLIF(CAST(bookable.sales_bookable AS DECIMAL(20, 4)), 0) AS DECIMAL(20, 4)) AS sales_occupancy

        FROM bookable 
            LEFT JOIN booked ON bookable.month = booked.month AND bookable.apartment_id = booked.apartment_id

        WHERE bookable.month >= DATE('2025-02-01')

        ORDER BY bookable.month DESC
    )

    SELECT month, apartment_id, booked, bookable, sales_bookable, occupancy, sales_occupancy FROM NEW
    UNION ALL
    SELECT month, apartment_id, booked, bookable, sales_bookable, occupancy, sales_occupancy FROM silver.finance.materialized_bookable_and_booked
"""

spark.sql(bookable_and_booked).write.mode("overwrite").saveAsTable("silver.finance.bookable_and_booked")

### 3.1.2. Deltas 7 Days Ago
###### (Bookable & Booked) 

In [0]:
bookable_and_booked_7_days_deltas = """
WITH 

    Base_Bookable AS (
        SELECT count(day) as bookable_1, trunc(day, 'month') month, apartment_id, guest_ready_date 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) occupancy
        GROUP BY trunc(day, 'month'), apartment_id, guest_ready_date
    )

    ,Blocked_not_occ AS (
        SELECT count(IS_BLOCKED) non_bookable, trunc(day, 'month') month, apartment_id 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) occupancy
        WHERE IS_BLOCKED 
                AND  (array_contains(BLOCK_REASON, 'RFBS')
                        OR  array_contains(BLOCK_REASON, 'MNTL')
                        OR  array_contains(BLOCK_REASON, 'QCHK')
                        OR  array_contains(BLOCK_REASON, 'PETS')
                        OR  array_contains(BLOCK_REASON, 'CLNG')
                        OR  array_contains(BLOCK_REASON, 'HOLD')
                        OR  array_contains(BLOCK_REASON, 'MNTE')
                        OR  array_contains(BLOCK_REASON, 'GRDT')
                        OR  array_contains(BLOCK_REASON, 'BTOB')
                        OR  array_contains(BLOCK_REASON, 'RLCT')
                        OR  array_contains(BLOCK_REASON, 'XOTH')
                        OR  array_contains(BLOCK_REASON, 'DSGN')
                )
                --AND CAST(BLOCK_REASON as string) <> '[SALS]'
        GROUP BY trunc(day, 'month'), apartment_id
    )

    ,Bookable_2 AS (
        SELECT 
            bb.month, bookable_1 as bookable, bookable_1 as sales_bookable, bb.apartment_id, guest_ready_date,
            JSON_TUPLE(CONTRACT_INFO, 'end_of_contract') e_o_contract
        FROM Base_Bookable bb 
            LEFT JOIN silver.backoffice.ams_contract_terms ON backoffice.ams_contract_terms.apartment_id = bb.apartment_id
        ORDER BY bb.month DESC
    )

    ,Bookable_4 AS (
        SELECT 
            month, apartment_id,
            CASE 
                WHEN month > trunc(e_o_contract, 'month') THEN 0
                WHEN month = trunc(e_o_contract, 'month') THEN EXTRACT(DAY FROM e_o_contract)::INT
                ELSE bookable
            END AS bookable,
            CASE 
                WHEN month < trunc(guest_ready_date, 'month') THEN 0
                WHEN month = trunc(guest_ready_date, 'month') THEN (EXTRACT(DAY FROM (trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY))::INT) - (EXTRACT(DAY FROM  guest_ready_date)::INT)
                WHEN month > trunc(e_o_contract, 'month') THEN 0
                WHEN month = trunc(e_o_contract, 'month') THEN EXTRACT(DAY FROM e_o_contract)::INT
                ELSE sales_bookable
            END AS sales_bookable, guest_ready_date

        FROM bookable_2
    )

    ,Bookable_3 AS (
        SELECT 
            bookable_4.month, bookable_4.apartment_id, bookable_4.bookable, 
            CASE WHEN (bookable_4.sales_bookable - COALESCE(non_bookable,0)) < 0 THEN 0
                ELSE (bookable_4.sales_bookable - COALESCE(non_bookable,0)) END AS sales_bookable

        FROM bookable_4 bookable_4
            LEFT JOIN Blocked_not_occ bno ON bookable_4.month = bno.month AND bookable_4.apartment_id = bno.apartment_id
    )

    ,Bookable AS (
        SELECT SUM(Bookable) Bookable, SUM(sales_bookable) sales_bookable, month, apartment_id FROM Bookable_3 GROUP BY month, apartment_id
    )
    ,Booked AS (
        SELECT count(IS_BOOKED) booked, trunc(day, 'month') month, apartment_id 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) occupancy 
        WHERE IS_BOOKED = true GROUP BY trunc(day, 'month'), apartment_id
    )

    ,New AS (
        SELECT 
            bookable.*, COALESCE(booked.booked,0) booked,
            CAST(CAST(COALESCE(booked.booked,0) AS DECIMAL(20, 4)) / NULLIF(CAST(bookable.bookable AS DECIMAL(20, 4)), 0) AS DECIMAL(20, 4)) AS occupancy,
            CAST(CAST(COALESCE(booked.booked,0) AS DECIMAL(20, 4)) / NULLIF(CAST(bookable.sales_bookable AS DECIMAL(20, 4)), 0) AS DECIMAL(20, 4)) AS sales_occupancy

        FROM bookable 
            LEFT JOIN booked ON bookable.month = booked.month AND bookable.apartment_id = booked.apartment_id

        ORDER BY bookable.month DESC
    )

    SELECT month, apartment_id, booked, bookable, sales_bookable, occupancy, sales_occupancy FROM NEW
"""

spark.sql(bookable_and_booked_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.bookable_and_booked_7_days_deltas")

### 3.1.3. Deltas Since Begining of Month
###### (Bookable & Booked)

In [0]:
bookable_and_booked_begining_month_deltas = """
WITH 
    Base_Bookable AS (
        SELECT count(day) as bookable_1, trunc(day, 'month') month, apartment_id, guest_ready_date 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) occupancy
        GROUP BY trunc(day, 'month'), apartment_id, guest_ready_date
    )

    ,Blocked_not_occ AS (
        SELECT count(IS_BLOCKED) non_bookable, trunc(day, 'month') month, apartment_id 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) occupancy
        WHERE IS_BLOCKED 
                AND  (array_contains(BLOCK_REASON, 'RFBS')
                        OR  array_contains(BLOCK_REASON, 'MNTL')
                        OR  array_contains(BLOCK_REASON, 'QCHK')
                        OR  array_contains(BLOCK_REASON, 'PETS')
                        OR  array_contains(BLOCK_REASON, 'CLNG')
                        OR  array_contains(BLOCK_REASON, 'HOLD')
                        OR  array_contains(BLOCK_REASON, 'MNTE')
                        OR  array_contains(BLOCK_REASON, 'GRDT')
                        OR  array_contains(BLOCK_REASON, 'BTOB')
                        OR  array_contains(BLOCK_REASON, 'RLCT')
                        OR  array_contains(BLOCK_REASON, 'XOTH')
                        OR  array_contains(BLOCK_REASON, 'DSGN')
                )
                --AND CAST(BLOCK_REASON as string) <> '[SALS]'
        GROUP BY trunc(day, 'month'), apartment_id
    )

    ,Bookable_2 AS (
        SELECT 
            bb.month, bookable_1 as bookable, bookable_1 as sales_bookable, bb.apartment_id, guest_ready_date,
            JSON_TUPLE(CONTRACT_INFO, 'end_of_contract') e_o_contract
        FROM Base_Bookable bb 
            LEFT JOIN silver.backoffice.ams_contract_terms ON backoffice.ams_contract_terms.apartment_id = bb.apartment_id
        ORDER BY bb.month DESC
    )

    ,Bookable_4 AS (
        SELECT 
            month, apartment_id,
            CASE 
                WHEN month > trunc(e_o_contract, 'month') THEN 0
                WHEN month = trunc(e_o_contract, 'month') THEN EXTRACT(DAY FROM e_o_contract)::INT
                ELSE bookable
            END AS bookable,
            CASE 
                WHEN month < trunc(guest_ready_date, 'month') THEN 0
                WHEN month = trunc(guest_ready_date, 'month') THEN (EXTRACT(DAY FROM (trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY))::INT) - (EXTRACT(DAY FROM  guest_ready_date)::INT)
                WHEN month > trunc(e_o_contract, 'month') THEN 0
                WHEN month = trunc(e_o_contract, 'month') THEN EXTRACT(DAY FROM e_o_contract)::INT
                ELSE sales_bookable
            END AS sales_bookable, guest_ready_date

        FROM bookable_2
    )

    ,Bookable_3 AS (
        SELECT 
            bookable_4.month, bookable_4.apartment_id, bookable_4.bookable, 
            CASE WHEN (bookable_4.sales_bookable - COALESCE(non_bookable,0)) < 0 THEN 0
                ELSE (bookable_4.sales_bookable - COALESCE(non_bookable,0)) END AS sales_bookable

        FROM bookable_4 bookable_4
            LEFT JOIN Blocked_not_occ bno ON bookable_4.month = bno.month AND bookable_4.apartment_id = bno.apartment_id
    )

    ,Bookable AS (
        SELECT SUM(Bookable) Bookable, SUM(sales_bookable) sales_bookable, month, apartment_id FROM Bookable_3 GROUP BY month, apartment_id
    )
    ,Booked AS (
        SELECT count(IS_BOOKED) booked, trunc(day, 'month') month, apartment_id 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) occupancy 
        WHERE IS_BOOKED = true GROUP BY trunc(day, 'month'), apartment_id
    )

    ,New AS (
        SELECT 
            bookable.*, COALESCE(booked.booked,0) booked,
            CAST(CAST(COALESCE(booked.booked,0) AS DECIMAL(20, 4)) / NULLIF(CAST(bookable.bookable AS DECIMAL(20, 4)), 0) AS DECIMAL(20, 4)) AS occupancy,
            CAST(CAST(COALESCE(booked.booked,0) AS DECIMAL(20, 4)) / NULLIF(CAST(bookable.sales_bookable AS DECIMAL(20, 4)), 0) AS DECIMAL(20, 4)) AS sales_occupancy

        FROM bookable 
            LEFT JOIN booked ON bookable.month = booked.month AND bookable.apartment_id = booked.apartment_id

        ORDER BY bookable.month DESC
    )

    SELECT month, apartment_id, booked, bookable, sales_bookable, occupancy, sales_occupancy FROM NEW
"""

spark.sql(bookable_and_booked_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.bookable_and_booked_begining_month_deltas")

# 

#4. VOLUME DATABASE

### 4.1. Actuals
###### (Volume Database)

In [0]:
volume_database = """
WITH 
    B AS (
        SELECT count(*), day, apartment_id, explode(block_reason) code  FROM silver.revenue.occupancy WHERE IS_BLOCKED GROUP BY day, apartment_id, block_reason
    )

    ,KI_4 AS (
        SELECT
            apartments.codename, apartments.id,
            apartments.city, 
            json_tuple(ams_contract_terms.contract_info, 'contract_signed') contract_sign_date,
            json_tuple(ams_contract_terms.contract_info, 'start_of_contract') start_of_contract,
            json_tuple(ams_contract_terms.contract_info, 'end_of_grace_period') end_of_carencia,
            json_tuple(ams_contract_terms.contract_info, 'end_of_contract') end_of_contract
        
        FROM silver.backoffice.apartments
            LEFT JOIN silver.backoffice.ams_contract_terms ON ams_contract_terms.apartment_id = apartments.id 
    )

    ,GR AS (
        SELECT value, x.apartment_id 
        FROM (SELECT MAX(updated_at) updated_at, apartment_id FROM silver.backoffice.apartment_dates WHERE type = 'guest_ready_date' GROUP BY apartment_id) x 
            LEFT JOIN silver.backoffice.apartment_dates ON apartment_dates.updated_at = x.updated_at AND apartment_dates.apartment_id = x.apartment_id
    )

    ,KI AS (
        SELECT 
            id, codename, city, contract_sign_date::DATE, start_of_contract::DATE, end_of_carencia::DATE, end_of_contract::DATE, value::DATE guest_ready_date
        
        FROM KI_4
            LEFT JOIN GR ON gr.apartment_id = KI_4.id
    )

    ,GS AS (
        SELECT
            explode(
                sequence(
                    DATE('2022-01-01'),
                    DATE('2028-01-01'),
                    INTERVAL 1 MONTH
                ) 
            ) month_date,
            codename, city, id, COALESCE(contract_sign_date, guest_ready_date) contract_sign_date, start_of_contract,
            guest_ready_date, end_of_carencia, end_of_contract
        
        FROM KI
    )

    ,JOINED AS (
        SELECT 
            month_date, city, codename, id,
            CASE WHEN trunc(start_of_contract, 'month') <= month_date THEN 1 ELSE 0 END AS contract_started,
            CASE WHEN trunc(contract_sign_date,'month') <= month_date THEN 1 ELSE 0 END AS contract_signed,
            CASE WHEN trunc(guest_ready_date,'month') <= month_date THEN 1 ELSE 0 END AS guest_ready,
            CASE WHEN trunc((end_of_contract+1),'month') <= month_date THEN 1 ELSE 0 END AS offboarded,
            CASE WHEN trunc(start_of_contract,'month') = month_date 
                        THEN (EXTRACT(DAY FROM trunc(month_date,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY') - EXTRACT(DAY FROM start_of_contract))::INTEGER
                WHEN trunc(start_of_contract,'month') - INTERVAL '1 MONTH' + INTERVAL '1 DAY' > month_date 
                        THEN 0
                ELSE EXTRACT(DAY FROM trunc(month_date,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')::INTEGER
            END AS days_paying_rent,
            CASE WHEN trunc(start_of_contract,'month') = month_date THEN 1- (EXTRACT(DAY FROM start_of_contract)::DECIMAL / (EXTRACT(DAY FROM (trunc(end_of_carencia, 'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')))::DECIMAL)
                WHEN (trunc(end_of_carencia,'month') > month_date AND trunc(start_of_contract,'month') <= month_date) THEN 1 
                WHEN trunc(end_of_carencia,'month') = month_date THEN EXTRACT(DAY FROM end_of_carencia)::DECIMAL / (EXTRACT(DAY FROM (trunc(end_of_carencia,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')))::DECIMAL
                ELSE 0 END AS grace_period,
            guest_ready_date, contract_sign_date, end_of_contract, start_of_contract
            
        FROM GS
    )

    ,Bookable AS (
        SELECT * FROM silver.finance.bookable_and_booked
    )    

    ,JOINED_2 AS (
        SELECT 
            JOINED.*, COALESCE(Bookable.booked, 0) occupied_nights, Bookable.Bookable, bookable.sales_bookable  
            
        FROM JOINED
            LEFT JOIN Bookable ON JOINED.month_date = Bookable.month AND JOINED.id = Bookable.apartment_id
    )

    ,Sales AS (
        SELECT
            check_in, check_out, (check_out-check_in)::INT los, sell_date, codename, id, type
        FROM silver.finance.new_sales_and_source AS S
    )

    ,Sales_2 AS (
        SELECT
            s.codename, s.id, MIN(b.check_in) AS check_in, MAX(b.check_out) AS check_out
        
        FROM Sales s
            LEFT JOIN silver.backoffice.bookings b ON s.id = b.id
            
        GROUP BY s.id, s.codename
    )

    ,bookings as (
        SELECT 
            trunc(sell_date, 'month') AS month,
            COUNT(CASE WHEN type = 'Extension' THEN 1 END) AS extension,
            COUNT(CASE WHEN type = 'Original Booking' THEN 1 END) AS regular_booking, codename
            
        FROM Sales
        GROUP BY trunc(sell_date, 'month'), codename
    )

    ,check_in as (
        SELECT 
            count(*) as check_in, trunc(check_in, 'month') AS month, codename
            
        FROM Sales_2
        
        GROUP BY trunc(check_in, 'month'), codename
    )

    ,check_out as (
        SELECT 
            count(*) as check_out, trunc(check_out, 'month') AS month, codename
            
        FROM Sales_2
        
        GROUP BY trunc(check_out, 'month'), codename
    )

    ,los as (
        SELECT 
            trunc(sell_date, 'month') AS month, codename, 
            SUM(CASE WHEN type = 'Extension' THEN los END) AS Nigths_Sold_extension,
            SUM(CASE WHEN type = 'Original Booking' THEN los END) AS Nigths_Sold_regular_booking
            
        FROM Sales
        GROUP BY trunc(sell_date, 'month'), codename
    )
        
    ,k as (
        SELECT 
            JOINED_2.*, extension, regular_booking, (regular_booking + extension) as Total_Bookings,
            CASE WHEN check_in IS NULL THEN 0 ELSE check_in END AS check_in,
            CASE WHEN check_out IS NULL THEN 0 ELSE check_out END AS check_out,
            CASE WHEN Nigths_Sold_extension IS NULL THEN 0 ELSE Nigths_Sold_extension END AS Nigths_Sold_extension, 
            CASE WHEN Nigths_Sold_regular_booking IS NULL THEN 0 ELSE Nigths_Sold_regular_booking END AS Nigths_Sold_regular_booking, 
            ((CASE WHEN Nigths_Sold_extension IS NULL THEN 0 ELSE Nigths_Sold_extension END) + (CASE WHEN Nigths_Sold_regular_booking IS NULL THEN 0 ELSE Nigths_Sold_regular_booking END)) as Nigths_Sold,
            EXTRACT(DAY FROM trunc(month_date, 'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY') as month_days
            
        FROM JOINED_2
            LEFT JOIN bookings ON bookings.month = JOINED_2.month_date AND bookings.codename = JOINED_2.codename
            LEFT JOIN check_in ON check_in.month = JOINED_2.month_date AND check_in.codename = JOINED_2.codename
            LEFT JOIN check_out ON check_out.month = JOINED_2.month_date AND check_out.codename = JOINED_2.codename
            LEFT JOIN los ON los.month = JOINED_2.month_date AND los.codename = JOINED_2.codename
        
        ORDER BY month_date ASC, guest_ready_date ASC
    )    
        
    ,Blocks as (
        SELECT 
            trunc(day, 'month') AS month,
            COUNT(CASE WHEN code = 'SALS' THEN 1 END) AS SALES_BLOCK_NIGHS,
            COUNT(CASE WHEN code = 'GRDT' THEN 1 END) AS GRD_BLOCK_NIGHS,
            COUNT(CASE WHEN code NOT IN ('GRDT', 'SALS') THEN 1 END) AS OTHER_BLOCK_NIGHS, apartment_id
            
        FROM B
        
        GROUP BY trunc(day,'month'), apartment_id
        
        ORDER BY trunc(day,'month') DESC   
    ) 

    ,New AS (
        SELECT 
            K.month_date, K.city, K.id, K.codename, K.month_days, K.contract_started, k.contract_signed, k.start_of_contract,
            CASE WHEN k.offboarded = 1 THEN 0 ELSE k.guest_ready END AS guest_ready,
            CASE WHEN k.contract_started = 0 THEN 0 ELSE k.offboarded END AS offboarded,
            k.days_paying_rent, 
            CASE WHEN k.Total_Bookings IS NULL THEN 0 ELSE k.Total_Bookings END AS Total_Bookings,
            CASE WHEN k.regular_booking IS NULL THEN 0 ELSE k.regular_booking END AS regular_booking,
            CASE WHEN k.extension IS NULL THEN 0 ELSE k.extension END AS extension,
            K.Nigths_Sold, K.Nigths_Sold_regular_booking, K.Nigths_Sold_extension, K.check_in, K.check_out,
            CASE WHEN k.occupied_nights IS NULL THEN 0 ELSE k.occupied_nights END AS occupied_nights, 
            k.Bookable, k.sales_Bookable, k.contract_sign_date, k.guest_ready_date, k.end_of_contract,
            CASE WHEN Blocks.SALES_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.SALES_BLOCK_NIGHS END AS SALES_BLOCK_NIGHS,
            CASE WHEN Blocks.GRD_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.GRD_BLOCK_NIGHS END AS GRD_BLOCK_NIGHS,
            CASE WHEN Blocks.OTHER_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.OTHER_BLOCK_NIGHS END AS OTHER_BLOCK_NIGHS
                
        FROM K
            LEFT JOIN Blocks ON Blocks.apartment_id = K.id AND Blocks.month = K.month_date
            
        ORDER BY month_date ASC, codename ASC
    )

    SELECT month_date, city, id, codename, month_days, contract_started, contract_signed, start_of_contract, 
    guest_ready, offboarded, days_paying_rent, Total_Bookings, regular_booking, extension, Nigths_Sold, 
    Nigths_Sold_regular_booking, Nigths_Sold_extension, check_in, check_out, occupied_nights, Bookable, 
    sales_Bookable, contract_sign_date, guest_ready_date, end_of_contract, SALES_BLOCK_NIGHS, GRD_BLOCK_NIGHS, 
    OTHER_BLOCK_NIGHS FROM New WHERE month_date >= DATE('2025-02-01') 

    UNION ALL

    SELECT month_date, city, id, codename, month_days, contract_started, contract_signed, start_of_contract, 
    guest_ready, offboarded, days_paying_rent, Total_Bookings, regular_booking, extension, Nigths_Sold, 
    Nigths_Sold_regular_booking, Nigths_Sold_extension, check_in, check_out, occupied_nights, Bookable, 
    sales_Bookable, contract_sign_date, guest_ready_date, end_of_contract, SALES_BLOCK_NIGHS, GRD_BLOCK_NIGHS, 
    OTHER_BLOCK_NIGHS FROM silver.finance.materialized_volume_database
    ORDER BY month_date ASC
"""

spark.sql(volume_database).write.mode("overwrite").saveAsTable("silver.finance.volume_database")

### 4.2. Deltas 7 Days Ago
###### (Volume Database)

In [0]:
volume_database_7_days_deltas = """
WITH 
    B AS (
        SELECT count(*), day, apartment_id, explode(block_reason) code 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) occupancy
        WHERE IS_BLOCKED GROUP BY day, apartment_id, block_reason
    )

    ,KI_4 AS (
        SELECT
            apartments.codename, apartments.id,
            apartments.city, 
            json_tuple(ams_contract_terms.contract_info, 'contract_signed') contract_sign_date,
            json_tuple(ams_contract_terms.contract_info, 'start_of_contract') start_of_contract,
            json_tuple(ams_contract_terms.contract_info, 'end_of_grace_period') end_of_carencia,
            json_tuple(ams_contract_terms.contract_info, 'end_of_contract') end_of_contract
        
        FROM silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments
            LEFT JOIN silver.backoffice.ams_contract_terms TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) ams_contract_terms ON ams_contract_terms.apartment_id = apartments.id 
    )

    ,GR AS (
        SELECT value, x.apartment_id 
        FROM (SELECT MAX(updated_at) updated_at, apartment_id 
              FROM silver.backoffice.apartment_dates TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartment_dates WHERE type = 'guest_ready_date' GROUP BY apartment_id) x 
              LEFT JOIN silver.backoffice.apartment_dates TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartment_dates ON apartment_dates.updated_at = x.updated_at AND apartment_dates.apartment_id = x.apartment_id
    )

    ,KI AS (
        SELECT 
            id, codename, city, contract_sign_date::DATE, start_of_contract::DATE, end_of_carencia::DATE, end_of_contract::DATE, value::DATE guest_ready_date
        
        FROM KI_4
            LEFT JOIN GR ON gr.apartment_id = KI_4.id
    )

    ,GS AS (
        SELECT
            explode(
                sequence(
                    DATE('2022-01-01'),
                    DATE('2028-01-01'),
                    INTERVAL 1 MONTH
                ) 
            ) month_date,
            codename, city, id, COALESCE(contract_sign_date, guest_ready_date) contract_sign_date, start_of_contract,
            guest_ready_date, end_of_carencia, end_of_contract
        
        FROM KI
    )

    ,JOINED AS (
        SELECT 
            month_date, city, codename, id,
            CASE WHEN trunc(start_of_contract, 'month') <= month_date THEN 1 ELSE 0 END AS contract_started,
            CASE WHEN trunc(contract_sign_date,'month') <= month_date THEN 1 ELSE 0 END AS contract_signed,
            CASE WHEN trunc(guest_ready_date,'month') <= month_date THEN 1 ELSE 0 END AS guest_ready,
            CASE WHEN trunc((end_of_contract+1),'month') <= month_date THEN 1 ELSE 0 END AS offboarded,
            CASE WHEN trunc(start_of_contract,'month') = month_date 
                        THEN (EXTRACT(DAY FROM trunc(month_date,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY') - EXTRACT(DAY FROM start_of_contract))::INTEGER
                WHEN trunc(start_of_contract,'month') - INTERVAL '1 MONTH' + INTERVAL '1 DAY' > month_date 
                        THEN 0
                ELSE EXTRACT(DAY FROM trunc(month_date,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')::INTEGER
            END AS days_paying_rent,
            CASE WHEN trunc(start_of_contract,'month') = month_date THEN 1- (EXTRACT(DAY FROM start_of_contract)::DECIMAL / (EXTRACT(DAY FROM (trunc(end_of_carencia, 'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')))::DECIMAL)
                WHEN (trunc(end_of_carencia,'month') > month_date AND trunc(start_of_contract,'month') <= month_date) THEN 1 
                WHEN trunc(end_of_carencia,'month') = month_date THEN EXTRACT(DAY FROM end_of_carencia)::DECIMAL / (EXTRACT(DAY FROM (trunc(end_of_carencia,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')))::DECIMAL
                ELSE 0 END AS grace_period,
            guest_ready_date, contract_sign_date, end_of_contract, start_of_contract
            
        FROM GS
    )

    ,Bookable AS (
        SELECT * FROM silver.finance.bookable_and_booked_7_days_deltas
    )    

    ,JOINED_2 AS (
        SELECT 
            JOINED.*, COALESCE(Bookable.booked, 0) occupied_nights, Bookable.Bookable, bookable.sales_bookable  
            
        FROM JOINED
            LEFT JOIN Bookable ON JOINED.month_date = Bookable.month AND JOINED.id = Bookable.apartment_id
    )

    ,Sales AS (
        SELECT
            check_in, check_out, (check_out-check_in)::INT los, sell_date, codename, id, type
        FROM silver.finance.new_sales_and_source_7_days_deltas AS S
    )

    ,Sales_2 AS (
        SELECT
            s.codename, s.id, MIN(b.check_in) AS check_in, MAX(b.check_out) AS check_out
        
        FROM Sales s
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) b ON s.id = b.id
            
        GROUP BY s.id, s.codename
    )

    ,bookings as (
        SELECT 
            trunc(sell_date, 'month') AS month,
            COUNT(CASE WHEN type = 'Extension' THEN 1 END) AS extension,
            COUNT(CASE WHEN type = 'Original Booking' THEN 1 END) AS regular_booking, codename
            
        FROM Sales
        GROUP BY trunc(sell_date, 'month'), codename
    )

    ,check_in as (
        SELECT 
            count(*) as check_in, trunc(check_in, 'month') AS month, codename
            
        FROM Sales_2
        
        GROUP BY trunc(check_in, 'month'), codename
    )

    ,check_out as (
        SELECT 
            count(*) as check_out, trunc(check_out, 'month') AS month, codename
            
        FROM Sales_2
        
        GROUP BY trunc(check_out, 'month'), codename
    )

    ,los as (
        SELECT 
            trunc(sell_date, 'month') AS month, codename, 
            SUM(CASE WHEN type = 'Extension' THEN los END) AS Nigths_Sold_extension,
            SUM(CASE WHEN type = 'Original Booking' THEN los END) AS Nigths_Sold_regular_booking
            
        FROM Sales
        GROUP BY trunc(sell_date, 'month'), codename
    )
        
    ,k as (
        SELECT 
            JOINED_2.*, extension, regular_booking, (regular_booking + extension) as Total_Bookings,
            CASE WHEN check_in IS NULL THEN 0 ELSE check_in END AS check_in,
            CASE WHEN check_out IS NULL THEN 0 ELSE check_out END AS check_out,
            CASE WHEN Nigths_Sold_extension IS NULL THEN 0 ELSE Nigths_Sold_extension END AS Nigths_Sold_extension, 
            CASE WHEN Nigths_Sold_regular_booking IS NULL THEN 0 ELSE Nigths_Sold_regular_booking END AS Nigths_Sold_regular_booking, 
            ((CASE WHEN Nigths_Sold_extension IS NULL THEN 0 ELSE Nigths_Sold_extension END) + (CASE WHEN Nigths_Sold_regular_booking IS NULL THEN 0 ELSE Nigths_Sold_regular_booking END)) as Nigths_Sold,
            EXTRACT(DAY FROM trunc(month_date, 'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY') as month_days
            
        FROM JOINED_2
            LEFT JOIN bookings ON bookings.month = JOINED_2.month_date AND bookings.codename = JOINED_2.codename
            LEFT JOIN check_in ON check_in.month = JOINED_2.month_date AND check_in.codename = JOINED_2.codename
            LEFT JOIN check_out ON check_out.month = JOINED_2.month_date AND check_out.codename = JOINED_2.codename
            LEFT JOIN los ON los.month = JOINED_2.month_date AND los.codename = JOINED_2.codename
        
        ORDER BY month_date ASC, guest_ready_date ASC
    )    
        
    ,Blocks as (
        SELECT 
            trunc(day, 'month') AS month,
            COUNT(CASE WHEN code = 'SALS' THEN 1 END) AS SALES_BLOCK_NIGHS,
            COUNT(CASE WHEN code = 'GRDT' THEN 1 END) AS GRD_BLOCK_NIGHS,
            COUNT(CASE WHEN code NOT IN ('GRDT', 'SALS') THEN 1 END) AS OTHER_BLOCK_NIGHS, apartment_id
            
        FROM B
        
        GROUP BY trunc(day,'month'), apartment_id
        
        ORDER BY trunc(day,'month') DESC   
    ) 

    ,New AS (
        SELECT 
            K.month_date, K.city, K.id, K.codename, K.month_days, K.contract_started, k.contract_signed, k.start_of_contract,
            CASE WHEN k.offboarded = 1 THEN 0 ELSE k.guest_ready END AS guest_ready,
            CASE WHEN k.contract_started = 0 THEN 0 ELSE k.offboarded END AS offboarded,
            k.days_paying_rent, 
            CASE WHEN k.Total_Bookings IS NULL THEN 0 ELSE k.Total_Bookings END AS Total_Bookings,
            CASE WHEN k.regular_booking IS NULL THEN 0 ELSE k.regular_booking END AS regular_booking,
            CASE WHEN k.extension IS NULL THEN 0 ELSE k.extension END AS extension,
            K.Nigths_Sold, K.Nigths_Sold_regular_booking, K.Nigths_Sold_extension, K.check_in, K.check_out,
            CASE WHEN k.occupied_nights IS NULL THEN 0 ELSE k.occupied_nights END AS occupied_nights, 
            k.Bookable, k.sales_Bookable, k.contract_sign_date, k.guest_ready_date, k.end_of_contract,
            CASE WHEN Blocks.SALES_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.SALES_BLOCK_NIGHS END AS SALES_BLOCK_NIGHS,
            CASE WHEN Blocks.GRD_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.GRD_BLOCK_NIGHS END AS GRD_BLOCK_NIGHS,
            CASE WHEN Blocks.OTHER_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.OTHER_BLOCK_NIGHS END AS OTHER_BLOCK_NIGHS
                
        FROM K
            LEFT JOIN Blocks ON Blocks.apartment_id = K.id AND Blocks.month = K.month_date
            
        ORDER BY month_date ASC, codename ASC
    )

    SELECT month_date, city, id, codename, month_days, contract_started, contract_signed, start_of_contract, 
    guest_ready, offboarded, days_paying_rent, Total_Bookings, regular_booking, extension, Nigths_Sold, 
    Nigths_Sold_regular_booking, Nigths_Sold_extension, check_in, check_out, occupied_nights, Bookable, 
    sales_Bookable, contract_sign_date, guest_ready_date, end_of_contract, SALES_BLOCK_NIGHS, GRD_BLOCK_NIGHS, 
    OTHER_BLOCK_NIGHS FROM New 
"""

spark.sql(volume_database_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.volume_database_7_days_deltas")

### 4.3. Deltas Since Begining of Month
###### (Volume Database)

In [0]:
volume_database_begining_month_deltas = """
WITH 
    B AS (
        SELECT count(*), day, apartment_id, explode(block_reason) code 
        FROM silver.revenue.occupancy TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) occupancy
        WHERE IS_BLOCKED GROUP BY day, apartment_id, block_reason
    )

    ,KI_4 AS (
        SELECT
            apartments.codename, apartments.id,
            apartments.city, 
            json_tuple(ams_contract_terms.contract_info, 'contract_signed') contract_sign_date,
            json_tuple(ams_contract_terms.contract_info, 'start_of_contract') start_of_contract,
            json_tuple(ams_contract_terms.contract_info, 'end_of_grace_period') end_of_carencia,
            json_tuple(ams_contract_terms.contract_info, 'end_of_contract') end_of_contract
        
        FROM silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments
            LEFT JOIN silver.backoffice.ams_contract_terms TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) ams_contract_terms ON ams_contract_terms.apartment_id = apartments.id 
    )

    ,GR AS (
        SELECT value, x.apartment_id 
        FROM (SELECT MAX(updated_at) updated_at, apartment_id 
              FROM silver.backoffice.apartment_dates TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartment_dates WHERE type = 'guest_ready_date' GROUP BY apartment_id) x 
              LEFT JOIN silver.backoffice.apartment_dates TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartment_dates ON apartment_dates.updated_at = x.updated_at AND apartment_dates.apartment_id = x.apartment_id
    )

    ,KI AS (
        SELECT 
            id, codename, city, contract_sign_date::DATE, start_of_contract::DATE, end_of_carencia::DATE, end_of_contract::DATE, value::DATE guest_ready_date
        
        FROM KI_4
            LEFT JOIN GR ON gr.apartment_id = KI_4.id
    )

    ,GS AS (
        SELECT
            explode(
                sequence(
                    DATE('2022-01-01'),
                    DATE('2028-01-01'),
                    INTERVAL 1 MONTH
                ) 
            ) month_date,
            codename, city, id, COALESCE(contract_sign_date, guest_ready_date) contract_sign_date, start_of_contract,
            guest_ready_date, end_of_carencia, end_of_contract
        
        FROM KI
    )

    ,JOINED AS (
        SELECT 
            month_date, city, codename, id,
            CASE WHEN trunc(start_of_contract, 'month') <= month_date THEN 1 ELSE 0 END AS contract_started,
            CASE WHEN trunc(contract_sign_date,'month') <= month_date THEN 1 ELSE 0 END AS contract_signed,
            CASE WHEN trunc(guest_ready_date,'month') <= month_date THEN 1 ELSE 0 END AS guest_ready,
            CASE WHEN trunc((end_of_contract+1),'month') <= month_date THEN 1 ELSE 0 END AS offboarded,
            CASE WHEN trunc(start_of_contract,'month') = month_date 
                        THEN (EXTRACT(DAY FROM trunc(month_date,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY') - EXTRACT(DAY FROM start_of_contract))::INTEGER
                WHEN trunc(start_of_contract,'month') - INTERVAL '1 MONTH' + INTERVAL '1 DAY' > month_date 
                        THEN 0
                ELSE EXTRACT(DAY FROM trunc(month_date,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')::INTEGER
            END AS days_paying_rent,
            CASE WHEN trunc(start_of_contract,'month') = month_date THEN 1- (EXTRACT(DAY FROM start_of_contract)::DECIMAL / (EXTRACT(DAY FROM (trunc(end_of_carencia, 'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')))::DECIMAL)
                WHEN (trunc(end_of_carencia,'month') > month_date AND trunc(start_of_contract,'month') <= month_date) THEN 1 
                WHEN trunc(end_of_carencia,'month') = month_date THEN EXTRACT(DAY FROM end_of_carencia)::DECIMAL / (EXTRACT(DAY FROM (trunc(end_of_carencia,'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY')))::DECIMAL
                ELSE 0 END AS grace_period,
            guest_ready_date, contract_sign_date, end_of_contract, start_of_contract
            
        FROM GS
    )

    ,Bookable AS (
        SELECT * FROM silver.finance.bookable_and_booked_begining_month_deltas
    )    

    ,JOINED_2 AS (
        SELECT 
            JOINED.*, COALESCE(Bookable.booked, 0) occupied_nights, Bookable.Bookable, bookable.sales_bookable  
            
        FROM JOINED
            LEFT JOIN Bookable ON JOINED.month_date = Bookable.month AND JOINED.id = Bookable.apartment_id
    )

    ,Sales AS (
        SELECT
            check_in, check_out, (check_out-check_in)::INT los, sell_date, codename, id, type
        FROM silver.finance.new_sales_and_source_begining_month_deltas AS S
    )

    ,Sales_2 AS (
        SELECT
            s.codename, s.id, MIN(b.check_in) AS check_in, MAX(b.check_out) AS check_out
        
        FROM Sales s
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) b ON s.id = b.id
            
        GROUP BY s.id, s.codename
    )

    ,bookings as (
        SELECT 
            trunc(sell_date, 'month') AS month,
            COUNT(CASE WHEN type = 'Extension' THEN 1 END) AS extension,
            COUNT(CASE WHEN type = 'Original Booking' THEN 1 END) AS regular_booking, codename
            
        FROM Sales
        GROUP BY trunc(sell_date, 'month'), codename
    )

    ,check_in as (
        SELECT 
            count(*) as check_in, trunc(check_in, 'month') AS month, codename
            
        FROM Sales_2
        
        GROUP BY trunc(check_in, 'month'), codename
    )

    ,check_out as (
        SELECT 
            count(*) as check_out, trunc(check_out, 'month') AS month, codename
            
        FROM Sales_2
        
        GROUP BY trunc(check_out, 'month'), codename
    )

    ,los as (
        SELECT 
            trunc(sell_date, 'month') AS month, codename, 
            SUM(CASE WHEN type = 'Extension' THEN los END) AS Nigths_Sold_extension,
            SUM(CASE WHEN type = 'Original Booking' THEN los END) AS Nigths_Sold_regular_booking
            
        FROM Sales
        GROUP BY trunc(sell_date, 'month'), codename
    )
        
    ,k as (
        SELECT 
            JOINED_2.*, extension, regular_booking, (regular_booking + extension) as Total_Bookings,
            CASE WHEN check_in IS NULL THEN 0 ELSE check_in END AS check_in,
            CASE WHEN check_out IS NULL THEN 0 ELSE check_out END AS check_out,
            CASE WHEN Nigths_Sold_extension IS NULL THEN 0 ELSE Nigths_Sold_extension END AS Nigths_Sold_extension, 
            CASE WHEN Nigths_Sold_regular_booking IS NULL THEN 0 ELSE Nigths_Sold_regular_booking END AS Nigths_Sold_regular_booking, 
            ((CASE WHEN Nigths_Sold_extension IS NULL THEN 0 ELSE Nigths_Sold_extension END) + (CASE WHEN Nigths_Sold_regular_booking IS NULL THEN 0 ELSE Nigths_Sold_regular_booking END)) as Nigths_Sold,
            EXTRACT(DAY FROM trunc(month_date, 'month') + INTERVAL '1 MONTH' - INTERVAL '1 DAY') as month_days
            
        FROM JOINED_2
            LEFT JOIN bookings ON bookings.month = JOINED_2.month_date AND bookings.codename = JOINED_2.codename
            LEFT JOIN check_in ON check_in.month = JOINED_2.month_date AND check_in.codename = JOINED_2.codename
            LEFT JOIN check_out ON check_out.month = JOINED_2.month_date AND check_out.codename = JOINED_2.codename
            LEFT JOIN los ON los.month = JOINED_2.month_date AND los.codename = JOINED_2.codename
        
        ORDER BY month_date ASC, guest_ready_date ASC
    )    
        
    ,Blocks as (
        SELECT 
            trunc(day, 'month') AS month,
            COUNT(CASE WHEN code = 'SALS' THEN 1 END) AS SALES_BLOCK_NIGHS,
            COUNT(CASE WHEN code = 'GRDT' THEN 1 END) AS GRD_BLOCK_NIGHS,
            COUNT(CASE WHEN code NOT IN ('GRDT', 'SALS') THEN 1 END) AS OTHER_BLOCK_NIGHS, apartment_id
            
        FROM B
        
        GROUP BY trunc(day,'month'), apartment_id
        
        ORDER BY trunc(day,'month') DESC   
    ) 

    ,New AS (
        SELECT 
            K.month_date, K.city, K.id, K.codename, K.month_days, K.contract_started, k.contract_signed, k.start_of_contract,
            CASE WHEN k.offboarded = 1 THEN 0 ELSE k.guest_ready END AS guest_ready,
            CASE WHEN k.contract_started = 0 THEN 0 ELSE k.offboarded END AS offboarded,
            k.days_paying_rent, 
            CASE WHEN k.Total_Bookings IS NULL THEN 0 ELSE k.Total_Bookings END AS Total_Bookings,
            CASE WHEN k.regular_booking IS NULL THEN 0 ELSE k.regular_booking END AS regular_booking,
            CASE WHEN k.extension IS NULL THEN 0 ELSE k.extension END AS extension,
            K.Nigths_Sold, K.Nigths_Sold_regular_booking, K.Nigths_Sold_extension, K.check_in, K.check_out,
            CASE WHEN k.occupied_nights IS NULL THEN 0 ELSE k.occupied_nights END AS occupied_nights, 
            k.Bookable, k.sales_Bookable, k.contract_sign_date, k.guest_ready_date, k.end_of_contract,
            CASE WHEN Blocks.SALES_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.SALES_BLOCK_NIGHS END AS SALES_BLOCK_NIGHS,
            CASE WHEN Blocks.GRD_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.GRD_BLOCK_NIGHS END AS GRD_BLOCK_NIGHS,
            CASE WHEN Blocks.OTHER_BLOCK_NIGHS IS NULL THEN 0 ELSE Blocks.OTHER_BLOCK_NIGHS END AS OTHER_BLOCK_NIGHS
                
        FROM K
            LEFT JOIN Blocks ON Blocks.apartment_id = K.id AND Blocks.month = K.month_date
            
        ORDER BY month_date ASC, codename ASC
    )

    SELECT month_date, city, id, codename, month_days, contract_started, contract_signed, start_of_contract, 
    guest_ready, offboarded, days_paying_rent, Total_Bookings, regular_booking, extension, Nigths_Sold, 
    Nigths_Sold_regular_booking, Nigths_Sold_extension, check_in, check_out, occupied_nights, Bookable, 
    sales_Bookable, contract_sign_date, guest_ready_date, end_of_contract, SALES_BLOCK_NIGHS, GRD_BLOCK_NIGHS, 
    OTHER_BLOCK_NIGHS FROM New 
"""

spark.sql(volume_database_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.volume_database_begining_month_deltas")